# .pt accessor tutorial

#### Sections
* [Price table accessor](#Price-table-accessor)
* [General](#General)
* [Interval](#Interval)
* [Timezone](#Timezone)
* [Reindexing](#Reindexing)
* [Downsampling](#Downsampling)
    * [`PTIntraday.downsample`](#PTIntraday.downsample)
    * [`PTDaily.downsample`](#PTDaily.downsample)
        * [Downsample to a multiple of days](#Downsample-to-a-multiple-of-days)
        * [Downsample to units longer than daily](#Downsample-to-units-longer-than-daily)
* [Indices trading status](#Indices-trading-status)
* [Querying prices](#Querying-prices)
* [Tidying data](#Tidying-data)
    * [`.operate`](#.operate)
* [`PTIntraday` session mapping](#PTIntraday-session-mapping)
* [`PTDailyIntradayComposite` methods](#PTDailyIntradayComposite-methods)

#### Note

The cell **outputs** shown in this tutorial are based on executing the cells at the time shown in the output of the following cell. Simply rerun the cells to bring any dynamic output up to date.

In [2]:
import pandas as pd
now = pd.Timestamp.now(tz="UTC").floor("T")
print(f"{now!r}")
print(f"{now.astimezone('America/New_York')!r}")

Timestamp('2022-05-13 16:14:00+0000', tz='UTC')
Timestamp('2022-05-13 12:14:00-0400', tz='America/New_York')


## Setup

Run the following cell to import tutorial dependencies.

In [3]:
from market_prices import PricesYahoo
import pytz
from market_prices.support import tutorial_helpers as th
from market_prices import helpers

Run the following cell to instantiate prices objects and define values used in this tutorial.

In [4]:
_prices = PricesYahoo("MSFT")
xnys = _prices.calendar_default
start_T1, end_T1 = th.get_sessions_range_for_bi(_prices, _prices.bis.T1)
start_H1 = th.get_sessions_range_for_bi(_prices, _prices.bis.H1)[0]
start_H1_oob = helpers.to_tz_naive(xnys.session_offset(start_H1, -2))

## Price table accessor

The `.pt` accessor provides access to a wealth of funcationality to interrogate and operate on `DataFrame` returned by `get`.

There are four `.pt` classes, each providing functionality tailored to a specific nature of price data:

* `pt.PTDaily` - daily price data (by session).
* `pt.PTIntraday` - intraday price data (including composite price tables where both intervals are intraday)
* `pt.PTMultipleSessions` - price data with an interval higher than one day.
* `pt.PTDailyIntradayComposite` - composite price data where earlier part of table has a daily interval and later part has an intraday interval.

Where not otherwise stated, properties and methods covered by this tutorial can be assumed as being available to all `.pt` classes.

Note:
* No `.pt` accessor property or method will operate on a price table in place, rather any operations are undertaken on a copy.
* The `.pt` accessor will not be available to a `DataFrame` returned from `get` in any of the following cases:
    * `side` is not None and `interval` is not daily (or inferred as daily).
    * `close_only` is passed as `True`.
* A `NotImplementedError` is raised if a `.pt` method is not available to a specific PT class.

Execute the following cells to set up a `DataFrame` for each of the above `.pt` classes. Each `DataFrame` is displayed for reference.

In [5]:
prices = PricesYahoo("MSFT, 9988.HK, AZN.L", lead_symbol="MSFT")

In [6]:
df_daily = prices.get(days=10)
df_daily

symbol            MSFT                                                \
                  open        high         low       close    volume   
2022-05-02  277.709991  284.940002  276.220001  284.470001  35151100   
2022-05-03  283.959991  284.130005  280.149994  281.779999  25978600   
2022-05-04  282.589996  290.880005  276.730011  289.980011  33599300   
2022-05-05  285.540009  286.350006  274.339996  277.350006  43260400   
2022-05-06  274.809998  279.250000  271.269989  274.730011  37748300   
2022-05-09  270.059998  272.359985  263.320007  264.579987  47726000   
2022-05-10  271.690002  273.750000  265.070007  269.500000  39336400   
2022-05-11  265.679993  271.359985  259.299988  260.549988  48975900   
2022-05-12  257.690002  259.880005  250.020004  255.350006  50970700   
2022-05-13  257.350006  262.369995  255.350006  262.309998  15644130   

symbol         9988.HK                                                  \
                  open        high        low       close       volume   
2022-05-02         NaN         NaN        NaN         NaN          NaN   
2022-05-03  100.000000  103.099998  92.500000  100.300003  103238870.0   
2022-05-04   98.500000   98.550003  95.849998   96.550003   29177225.0   
2022-05-05   99.500000  101.099998  96.199997   96.699997   27991225.0   
2022-05-06   92.050003   92.849998  89.300003   90.349998   50887668.0   
2022-05-09         NaN         NaN        NaN         NaN          NaN   
2022-05-10   83.500000   87.199997  83.150002   86.000000   64139439.0   
2022-05-11   83.699997   87.949997  83.599998   85.650002   63211009.0   
2022-05-12   81.849998   82.550003  79.900002   80.000000   69966835.0   
2022-05-13   82.199997   82.849998  81.250000   82.199997   53688518.0   

symbol        AZN.L                                            
               open         high      low    close     volume  
2022-05-02      NaN          NaN      NaN      NaN        NaN  
2022-05-03  10498.0  10710.00000  10472.0  10684.0  2234172.0  
2022-05-04  10792.0  10792.00000  10458.0  10506.0  2213731.0  
2022-05-05  10532.0  10632.00000  10478.0  10512.0  2501672.0  
2022-05-06  10464.0  10474.00000  10222.0  10322.0  2591818.0  
2022-05-09  10372.0  10462.47168  10118.0  10140.0  2216925.0  
2022-05-10  10166.0  10338.00000  10014.0  10246.0  1759026.0  
2022-05-11  10210.0  10262.00000   9901.0  10026.0  4552005.0  
2022-05-12   9948.0   9984.00000   9764.0   9965.0  3585647.0  
2022-05-13  10054.0  10300.00000   9996.0  10286.0  2084381.0

In [7]:
df_daily.pt

In [8]:
df_intraday = prices.get(days=2)
df_intraday

symbol                                            MSFT              \
                                                  open        high   
[2022-05-11 12:15:00, 2022-05-11 12:20:00)  265.250000  265.522614   
[2022-05-11 12:20:00, 2022-05-11 12:25:00)  264.790009  266.299988   
[2022-05-11 12:25:00, 2022-05-11 12:30:00)  266.160004  266.250000   
[2022-05-11 12:30:00, 2022-05-11 12:35:00)  264.329987  264.769989   
[2022-05-11 12:35:00, 2022-05-11 12:40:00)  264.440002  265.329987   
...                                                ...         ...   
[2022-05-13 11:50:00, 2022-05-13 11:55:00)  261.600006  261.915009   
[2022-05-13 11:55:00, 2022-05-13 12:00:00)  261.839996  261.855011   
[2022-05-13 12:00:00, 2022-05-13 12:05:00)  261.339996  261.619995   
[2022-05-13 12:05:00, 2022-05-13 12:10:00)  261.589996  262.019989   
[2022-05-13 12:10:00, 2022-05-13 12:15:00)  262.024994  262.369995   

symbol                                                                        \
                                                   low       close    volume   
[2022-05-11 12:15:00, 2022-05-11 12:20:00)  264.700012  264.809998  195612.0   
[2022-05-11 12:20:00, 2022-05-11 12:25:00)  264.709991  266.200012  281529.0   
[2022-05-11 12:25:00, 2022-05-11 12:30:00)  264.130005  264.359985  398680.0   
[2022-05-11 12:30:00, 2022-05-11 12:35:00)  264.140015  264.480011  301410.0   
[2022-05-11 12:35:00, 2022-05-11 12:40:00)  264.350006  264.730011  369477.0   
...                                                ...         ...       ...   
[2022-05-13 11:50:00, 2022-05-13 11:55:00)  261.459991  261.859985  199111.0   
[2022-05-13 11:55:00, 2022-05-13 12:00:00)  261.149994  261.309998  202250.0   
[2022-05-13 12:00:00, 2022-05-13 12:05:00)  261.170013  261.600006  179609.0   
[2022-05-13 12:05:00, 2022-05-13 12:10:00)  261.369995  262.019989  288129.0   
[2022-05-13 12:10:00, 2022-05-13 12:15:00)  261.790009  262.359985  182675.0   

symbol                                     9988.HK                        \
                                              open high low close volume   
[2022-05-11 12:15:00, 2022-05-11 12:20:00)     NaN  NaN NaN   NaN    NaN   
[2022-05-11 12:20:00, 2022-05-11 12:25:00)     NaN  NaN NaN   NaN    NaN   
[2022-05-11 12:25:00, 2022-05-11 12:30:00)     NaN  NaN NaN   NaN    NaN   
[2022-05-11 12:30:00, 2022-05-11 12:35:00)     NaN  NaN NaN   NaN    NaN   
[2022-05-11 12:35:00, 2022-05-11 12:40:00)     NaN  NaN NaN   NaN    NaN   
...                                            ...  ...  ..   ...    ...   
[2022-05-13 11:50:00, 2022-05-13 11:55:00)     NaN  NaN NaN   NaN    NaN   
[2022-05-13 11:55:00, 2022-05-13 12:00:00)     NaN  NaN NaN   NaN    NaN   
[2022-05-13 12:00:00, 2022-05-13 12:05:00)     NaN  NaN NaN   NaN    NaN   
[2022-05-13 12:05:00, 2022-05-13 12:10:00)     NaN  NaN NaN   NaN    NaN   
[2022-05-13 12:10:00, 2022-05-13 12:15:00)     NaN  NaN NaN   NaN    NaN   

symbol                                     AZN.L                        
                                            open high low close volume  
[2022-05-11 12:15:00, 2022-05-11 12:20:00)   NaN  NaN NaN   NaN    NaN  
[2022-05-11 12:20:00, 2022-05-11 12:25:00)   NaN  NaN NaN   NaN    NaN  
[2022-05-11 12:25:00, 2022-05-11 12:30:00)   NaN  NaN NaN   NaN    NaN  
[2022-05-11 12:30:00, 2022-05-11 12:35:00)   NaN  NaN NaN   NaN    NaN  
[2022-05-11 12:35:00, 2022-05-11 12:40:00)   NaN  NaN NaN   NaN    NaN  
...                                          ...  ...  ..   ...    ...  
[2022-05-13 11:50:00, 2022-05-13 11:55:00)   NaN  NaN NaN   NaN    NaN  
[2022-05-13 11:55:00, 2022-05-13 12:00:00)   NaN  NaN NaN   NaN    NaN  
[2022-05-13 12:00:00, 2022-05-13 12:05:00)   NaN  NaN NaN   NaN    NaN  
[2022-05-13 12:05:00, 2022-05-13 12:10:00)   NaN  NaN NaN   NaN    NaN  
[2022-05-13 12:10:00, 2022-05-13 12:15:00)   NaN  NaN NaN   NaN    NaN  

[420 rows x 15 columns]

In [9]:
df_intraday.pt

In [10]:
df_mult_days = prices.get("3D", months=1)
df_mult_days

symbol                          MSFT                                      \
                                open        high         low       close   
[2022-04-14, 2022-04-20)  288.089996  288.309998  278.339996  285.299988   
[2022-04-20, 2022-04-25)  289.399994  293.299988  273.380005  274.029999   
[2022-04-25, 2022-04-28)  273.290009  290.970001  270.000000  283.220001   
[2022-04-28, 2022-05-03)  285.190002  290.980011  276.220001  284.470001   
[2022-05-03, 2022-05-06)  283.959991  290.880005  274.339996  277.350006   
[2022-05-06, 2022-05-11)  274.809998  279.250000  263.320007  269.500000   
[2022-05-11, 2022-05-14)  265.679993  271.359985  250.020004  262.309998   

symbol                                    9988.HK                         \
                               volume        open        high        low   
[2022-04-14, 2022-04-20)   71297300.0   97.650002   98.000000  90.900002   
[2022-04-20, 2022-04-25)   81767100.0   91.099998   92.949997  83.400002   
[2022-04-25, 2022-04-28)  145675000.0   84.800003   87.900002  81.199997   
[2022-04-28, 2022-05-03)  105822700.0   88.000000  103.800003  86.300003   
[2022-05-03, 2022-05-06)  102838300.0  100.000000  103.099998  92.500000   
[2022-05-06, 2022-05-11)  124810700.0   92.050003   92.849998  83.150002   
[2022-05-11, 2022-05-14)  115590730.0   83.699997   87.949997  79.900002   

symbol                                               AZN.L                    \
                               close       volume     open     high      low   
[2022-04-14, 2022-04-20)   91.500000   86409647.0  10432.0  10582.0  10392.0   
[2022-04-20, 2022-04-25)   86.650002   95085703.0  10394.0  10606.0  10260.0   
[2022-04-25, 2022-04-28)   84.550003  109223954.0  10144.0  10738.0  10090.0   
[2022-04-28, 2022-05-03)  102.099998  142809591.0  10550.0  10762.0  10422.0   
[2022-05-03, 2022-05-06)   96.699997  160407320.0  10498.0  10792.0  10458.0   
[2022-05-06, 2022-05-11)   86.000000  115027107.0  10464.0  10474.0  10014.0   
[2022-05-11, 2022-05-14)   82.199997  186866362.0  10210.0  10300.0   9764.0   

symbol                                         
                            close      volume  
[2022-04-14, 2022-04-20)  10500.0   5240738.0  
[2022-04-20, 2022-04-25)  10280.0  10308895.0  
[2022-04-25, 2022-04-28)  10512.0  10811661.0  
[2022-04-28, 2022-05-03)  10688.0   5516054.0  
[2022-05-03, 2022-05-06)  10512.0   6949575.0  
[2022-05-06, 2022-05-11)  10246.0   6567769.0  
[2022-05-11, 2022-05-14)  10286.0  10222033.0

In [11]:
df_mult_days.pt

In [12]:
end = xnys.session_close(end_T1) - pd.Timedelta(3, "T")
df_comp = prices.get(start=start_H1_oob, end=end, composite=True)
df_comp

symbol                                            MSFT              \
                                                  open        high   
[2020-05-12, 2020-05-12)                    186.800003  187.039993   
[2020-05-13, 2020-05-13)                    182.550003  184.050003   
[2020-05-14, 2020-05-14)                    177.539993  180.690002   
[2020-05-15, 2020-05-15)                    179.059998  187.059998   
[2020-05-18, 2020-05-18)                    185.750000  186.199997   
...                                                ...         ...   
[2022-05-12 19:52:00, 2022-05-12 19:53:00)  254.529999  254.630005   
[2022-05-12 19:53:00, 2022-05-12 19:54:00)  254.550003  254.559998   
[2022-05-12 19:54:00, 2022-05-12 19:55:00)  254.029999  254.335007   
[2022-05-12 19:55:00, 2022-05-12 19:56:00)  254.000000  254.309998   
[2022-05-12 19:56:00, 2022-05-12 19:57:00)  254.080002  254.899994   

symbol                                                              \
                                                   low       close   
[2020-05-12, 2020-05-12)                    182.300003  182.509995   
[2020-05-13, 2020-05-13)                    176.539993  179.750000   
[2020-05-14, 2020-05-14)                    175.679993  180.529999   
[2020-05-15, 2020-05-15)                    177.000000  183.160004   
[2020-05-18, 2020-05-18)                    183.960007  184.910004   
...                                                ...         ...   
[2022-05-12 19:52:00, 2022-05-12 19:53:00)  254.279999  254.529999   
[2022-05-12 19:53:00, 2022-05-12 19:54:00)  253.740005  254.029999   
[2022-05-12 19:54:00, 2022-05-12 19:55:00)  253.259995  254.020004   
[2022-05-12 19:55:00, 2022-05-12 19:56:00)  253.820007  254.070007   
[2022-05-12 19:56:00, 2022-05-12 19:57:00)  254.080002  254.800003   

symbol                                                     9988.HK  \
                                                volume        open   
[2020-05-12, 2020-05-12)                    32038200.0  198.899994   
[2020-05-13, 2020-05-13)                    44711500.0  195.500000   
[2020-05-14, 2020-05-14)                    41873900.0  194.500000   
[2020-05-15, 2020-05-15)                    46610400.0  195.000000   
[2020-05-18, 2020-05-18)                    35264500.0  198.300003   
...                                                ...         ...   
[2022-05-12 19:52:00, 2022-05-12 19:53:00)    219682.0         NaN   
[2022-05-12 19:53:00, 2022-05-12 19:54:00)    209035.0         NaN   
[2022-05-12 19:54:00, 2022-05-12 19:55:00)    277838.0         NaN   
[2022-05-12 19:55:00, 2022-05-12 19:56:00)    289084.0         NaN   
[2022-05-12 19:56:00, 2022-05-12 19:57:00)    352067.0         NaN   

symbol                                                              \
                                                  high         low   
[2020-05-12, 2020-05-12)                    199.699997  197.199997   
[2020-05-13, 2020-05-13)                    197.399994  194.300003   
[2020-05-14, 2020-05-14)                    195.899994  194.100006   
[2020-05-15, 2020-05-15)                    197.100006  194.100006   
[2020-05-18, 2020-05-18)                    203.000000  197.399994   
...                                                ...         ...   
[2022-05-12 19:52:00, 2022-05-12 19:53:00)         NaN         NaN   
[2022-05-12 19:53:00, 2022-05-12 19:54:00)         NaN         NaN   
[2022-05-12 19:54:00, 2022-05-12 19:55:00)         NaN         NaN   
[2022-05-12 19:55:00, 2022-05-12 19:56:00)         NaN         NaN   
[2022-05-12 19:56:00, 2022-05-12 19:57:00)         NaN         NaN   

symbol                                                               AZN.L  \
                                                 close      volume    open   
[2020-05-12, 2020-05-12)                    199.699997  14037259.0  8690.0   
[2020-05-13, 2020-05-13)                    196.300003  26074457.0  8800.0   
[2020-05-14, 2020-05-14)                    

In [13]:
df_comp.pt

In [14]:
dfs = (df_daily, df_intraday, df_mult_days, df_comp)

## General

The `.pt.has_symbols` and `.pt.symbols` properties provide for interrogating the table's symbols.

In [15]:
df_daily.pt.has_symbols

True

In [16]:
df_intraday.pt.symbols

['MSFT', '9988.HK', 'AZN.L']

`.pt.first_ts` and `.pt.last_ts` return the, respectively, left side of the first indice and right side of the last indice.

In [17]:
df_intraday.pt.first_ts, df_intraday.pt.last_ts

(Timestamp('2022-05-11 12:15:00-0400', tz='America/New_York'),
 Timestamp('2022-05-13 12:15:00-0400', tz='America/New_York'))

...except when prices are daily, in which case they simply return first and last indice.

In [18]:
df_daily.pt.first_ts, df_daily.pt.last_ts

(Timestamp('2022-05-02 00:00:00', freq='B'),
 Timestamp('2022-05-13 00:00:00', freq='B'))

## Interval

* `.pt.has_regular_interval` queries if the table has a regular interval.
* `.pt.interval` will return the table interval (or None if the table interval is irregular or cannot be ascertained).
* `.pt.is_intraday` and `.pt.is_daily` query if the interval is, respectively, daily (one day) or intraday.

In [19]:
for df in dfs:
    print(df.pt.__class__.__name__)
    print(f"{df.pt.has_regular_interval=}")
    print(f"{df.pt.interval=}")
    print(f"{df.pt.is_intraday=}")
    print(f"{df.pt.is_daily=}\n")

PTDaily
df.pt.has_regular_interval=True
df.pt.interval=<TDInterval.D1: Timedelta('1 days 00:00:00')>
df.pt.is_intraday=False
df.pt.is_daily=True

PTIntraday
df.pt.has_regular_interval=True
df.pt.interval=<TDInterval.T5: Timedelta('0 days 00:05:00')>
df.pt.is_intraday=True
df.pt.is_daily=False

PTMultipleSessions
df.pt.has_regular_interval=False
df.pt.interval=None
df.pt.is_intraday=False
df.pt.is_daily=False

PTDailyIntradayComposite
df.pt.has_regular_interval=False
df.pt.interval=None
df.pt.is_intraday=False
df.pt.is_daily=False



Note how the interval cannot be ascertained for the multiple sessions and composite tables.

Unique to the PTIntraday class, the following methods can be used to query the number and regularity of trading minutes associated with indices.

* `.pt.indices_trading_minutes`
* `.pt.indices_trading_minutes_values`
* `.pt.trading_minutes_interval`
* `.pt.indices_have_regular_trading_minutes`
* `.pt.indices_length`
* `.pt.by_indice_length`

Explanation and examples of these methods is offered in the [Interrogating indices with .pt accessor](./anchor.ipynb#Interrogating-indices-with-.pt-accessor) section of the [anchors](./anchors.ipynb) tutorial, with the exception of `.pt.indices_trading_minutes_values`.

`.pt.indices_trading_minutes_values` returns a set of trading minutes associated with the indices. The trading minutes are evaluated against a specific calendar...

In [20]:
df_intraday.pt.indices_trading_minutes_values(xnys)

array([5, 0], dtype=int64)

In this case all indices cover either 5 trading minutes (if they fall within a New York session) or 0 (if they fall outside of a New York session). 

## Timezone

`.pt.tz` returns the index's timezone, or `None` if index is not timezone-aware.

In [21]:
for df in dfs:
    print(df.pt.__class__.__name__)
    print(f"{df.pt.tz=}\n")

PTDaily
df.pt.tz=None

PTIntraday
df.pt.tz=<DstTzInfo 'America/New_York' LMT-1 day, 19:04:00 STD>

PTMultipleSessions
df.pt.tz=None

PTDailyIntradayComposite
df.pt.tz=<UTC>



Note that, With the exception of PTIntraday, all price tables can only be either timezone-naive or have timezone as UTC.

`.pt.naive` returns a copy of the table with the index as timezone-naive.

In [22]:
df_intraday.pt.naive.pt.tz == None

True

Notice how the above chains calls together, with the accessor called on the return from `pt.naive`.

`.pt.utc` returns a copy with the index converted to UTC. Compare the first two rows of the `df_intraday` table...

In [23]:
df_intraday[:2]

symbol                                            MSFT              \
                                                  open        high   
[2022-05-11 12:15:00, 2022-05-11 12:20:00)  265.250000  265.522614   
[2022-05-11 12:20:00, 2022-05-11 12:25:00)  264.790009  266.299988   

symbol                                                                        \
                                                   low       close    volume   
[2022-05-11 12:15:00, 2022-05-11 12:20:00)  264.700012  264.809998  195612.0   
[2022-05-11 12:20:00, 2022-05-11 12:25:00)  264.709991  266.200012  281529.0   

symbol                                     9988.HK                        \
                                              open high low close volume   
[2022-05-11 12:15:00, 2022-05-11 12:20:00)     NaN  NaN NaN   NaN    NaN   
[2022-05-11 12:20:00, 2022-05-11 12:25:00)     NaN  NaN NaN   NaN    NaN   

symbol                                     AZN.L                        
                                            open high low close volume  
[2022-05-11 12:15:00, 2022-05-11 12:20:00)   NaN  NaN NaN   NaN    NaN  
[2022-05-11 12:20:00, 2022-05-11 12:25:00)   NaN  NaN NaN   NaN    NaN

In [24]:
df_intraday.pt.utc[:2]

symbol                                            MSFT              \
                                                  open        high   
[2022-05-11 16:15:00, 2022-05-11 16:20:00)  265.250000  265.522614   
[2022-05-11 16:20:00, 2022-05-11 16:25:00)  264.790009  266.299988   

symbol                                                                        \
                                                   low       close    volume   
[2022-05-11 16:15:00, 2022-05-11 16:20:00)  264.700012  264.809998  195612.0   
[2022-05-11 16:20:00, 2022-05-11 16:25:00)  264.709991  266.200012  281529.0   

symbol                                     9988.HK                        \
                                              open high low close volume   
[2022-05-11 16:15:00, 2022-05-11 16:20:00)     NaN  NaN NaN   NaN    NaN   
[2022-05-11 16:20:00, 2022-05-11 16:25:00)     NaN  NaN NaN   NaN    NaN   

symbol                                     AZN.L                        
                                            open high low close volume  
[2022-05-11 16:15:00, 2022-05-11 16:20:00)   NaN  NaN NaN   NaN    NaN  
[2022-05-11 16:20:00, 2022-05-11 16:25:00)   NaN  NaN NaN   NaN    NaN

`.pt.set_tz` can be used to return a copy of a `PTIntraday` table set to a different timezone.

In [25]:
df_intraday.pt.set_tz("Asia/Hong_Kong")[:2]

symbol                                            MSFT              \
                                                  open        high   
[2022-05-12 00:15:00, 2022-05-12 00:20:00)  265.250000  265.522614   
[2022-05-12 00:20:00, 2022-05-12 00:25:00)  264.790009  266.299988   

symbol                                                                        \
                                                   low       close    volume   
[2022-05-12 00:15:00, 2022-05-12 00:20:00)  264.700012  264.809998  195612.0   
[2022-05-12 00:20:00, 2022-05-12 00:25:00)  264.709991  266.200012  281529.0   

symbol                                     9988.HK                        \
                                              open high low close volume   
[2022-05-12 00:15:00, 2022-05-12 00:20:00)     NaN  NaN NaN   NaN    NaN   
[2022-05-12 00:20:00, 2022-05-12 00:25:00)     NaN  NaN NaN   NaN    NaN   

symbol                                     AZN.L                        
                                            open high low close volume  
[2022-05-12 00:15:00, 2022-05-12 00:20:00)   NaN  NaN NaN   NaN    NaN  
[2022-05-12 00:20:00, 2022-05-12 00:25:00)   NaN  NaN NaN   NaN    NaN

Notice that in all the above examples the price data is the same, only the index timezone changes.

The helper method `.pt.convert_to_table_tz` will convert a Timestamp to the same timezone as the table.

In [26]:
ts = pd.Timestamp("2022-05-10 19:22", tz=pytz.UTC)
df_intraday.pt.convert_to_table_tz(ts)

Timestamp('2022-05-10 15:22:00-0400', tz='America/New_York')

## Reindexing

`.pt.reindex_to_calendar` is available to `PTDaily` and `PTIntraday` to return a copy of the table reindexed to a trading index evaluated against a specific calendar.

For example, to reindex to only indices corresponding to the Hong Kong exchange...

In [27]:
xhkg = prices.calendars["9988.HK"]
df_intraday.pt.reindex_to_calendar(xhkg)

symbol                                     MSFT                        \
                                           open high low close volume   
[2022-05-11 21:30:00, 2022-05-11 21:35:00)  NaN  NaN NaN   NaN    NaN   
[2022-05-11 21:35:00, 2022-05-11 21:40:00)  NaN  NaN NaN   NaN    NaN   
[2022-05-11 21:40:00, 2022-05-11 21:45:00)  NaN  NaN NaN   NaN    NaN   
[2022-05-11 21:45:00, 2022-05-11 21:50:00)  NaN  NaN NaN   NaN    NaN   
[2022-05-11 21:50:00, 2022-05-11 21:55:00)  NaN  NaN NaN   NaN    NaN   
...                                         ...  ...  ..   ...    ...   
[2022-05-13 03:35:00, 2022-05-13 03:40:00)  NaN  NaN NaN   NaN    NaN   
[2022-05-13 03:40:00, 2022-05-13 03:45:00)  NaN  NaN NaN   NaN    NaN   
[2022-05-13 03:45:00, 2022-05-13 03:50:00)  NaN  NaN NaN   NaN    NaN   
[2022-05-13 03:50:00, 2022-05-13 03:55:00)  NaN  NaN NaN   NaN    NaN   
[2022-05-13 03:55:00, 2022-05-13 04:00:00)  NaN  NaN NaN   NaN    NaN   

symbol                                        9988.HK                        \
                                                 open       high        low   
[2022-05-11 21:30:00, 2022-05-11 21:35:00)  81.849998  82.550003  81.599998   
[2022-05-11 21:35:00, 2022-05-11 21:40:00)  81.800003  81.949997  81.199997   
[2022-05-11 21:40:00, 2022-05-11 21:45:00)  81.800003  82.300003  81.750000   
[2022-05-11 21:45:00, 2022-05-11 21:50:00)  82.300003  82.500000  82.150002   
[2022-05-11 21:50:00, 2022-05-11 21:55:00)  82.250000  82.349998  81.849998   
...                                               ...        ...        ...   
[2022-05-13 03:35:00, 2022-05-13 03:40:00)  82.449997  82.449997  82.150002   
[2022-05-13 03:40:00, 2022-05-13 03:45:00)  82.300003  82.300003  82.250000   
[2022-05-13 03:45:00, 2022-05-13 03:50:00)  82.250000  82.449997  82.250000   
[2022-05-13 03:50:00, 2022-05-13 03:55:00)  82.449997  82.449997  82.349998   
[2022-05-13 03:55:00, 2022-05-13 04:00:00)  82.349998  82.500000  80.000000   

symbol                                                            \
                                                close     volume   
[2022-05-11 21:30:00, 2022-05-11 21:35:00)  81.750000        0.0   
[2022-05-11 21:35:00, 2022-05-11 21:40:00)  81.800003  1931829.0   
[2022-05-11 21:40:00, 2022-05-11 21:45:00)  82.300003  1524906.0   
[2022-05-11 21:45:00, 2022-05-11 21:50:00)  82.300003  1075405.0   
[2022-05-11 21:50:00, 2022-05-11 21:55:00)  81.849998  1714921.0   
...                                               ...        ...   
[2022-05-13 03:35:00, 2022-05-13 03:40:00)  82.250000  1038650.0   
[2022-05-13 03:40:00, 2022-05-13 03:45:00)  82.300003   719180.0   
[2022-05-13 03:45:00, 2022-05-13 03:50:00)  82.449997  1284602.0   
[2022-05-13 03:50:00, 2022-05-13 03:55:00)  82.400002  1091340.0   
[2022-05-13 03:55:00, 2022-05-13 04:00:00)  82.199997  1590700.0   

symbol                                             AZN.L                \
                                                    open          high   
[2022-05-11 21:30:00, 2022-05-11 21:35:00)           NaN           NaN   
[2022-05-11 21:35:00, 2022-05-11 21:40:00)           NaN           NaN   
[2022-05-11 21:40:00, 2022-05-11 21:45:00)           NaN           NaN   
[2022-05-11 21:45:00, 2022-05-11 21:50:00)           NaN           NaN   
[2022-05-11 21:50:00, 2022-05-11 21:55:00)           NaN           NaN   
...                                                  ...           ...   
[2022-05-13 03:35:00, 2022-05-13 03:40:00)  10112.000000  10140.000000   
[2022-05-13 03:40:00, 2022-05-13 03:45:00)  10144.000000  10158.000000   
[2022-05-13 03:45:00, 2022-05-13 03:50:00)  10134.000000  10156.000000   
[2022-05-13 03:50:00, 2022-05-13 03:55:00)  10146.000000  10152.692383   
[2022-05-13 03:55:00, 2022-05-13 04:00:00)  10142.410156  10158.000000   

symbol                                                                      
                                                low         close   volume  
[2022-05-11 21:30:00

Notice that all prices for MSFT are now missing as the New York exchange is always closed when the Hong Kong exchange is open. Although they are usually open on the same days...

In [28]:
df_daily.pt.reindex_to_calendar(xhkg)

symbol            MSFT                                                \
                  open        high         low       close    volume   
2022-05-03  283.959991  284.130005  280.149994  281.779999  25978600   
2022-05-04  282.589996  290.880005  276.730011  289.980011  33599300   
2022-05-05  285.540009  286.350006  274.339996  277.350006  43260400   
2022-05-06  274.809998  279.250000  271.269989  274.730011  37748300   
2022-05-10  271.690002  273.750000  265.070007  269.500000  39336400   
2022-05-11  265.679993  271.359985  259.299988  260.549988  48975900   
2022-05-12  257.690002  259.880005  250.020004  255.350006  50970700   
2022-05-13  257.350006  262.369995  255.350006  262.309998  15644130   

symbol         9988.HK                                                  \
                  open        high        low       close       volume   
2022-05-03  100.000000  103.099998  92.500000  100.300003  103238870.0   
2022-05-04   98.500000   98.550003  95.849998   96.550003   29177225.0   
2022-05-05   99.500000  101.099998  96.199997   96.699997   27991225.0   
2022-05-06   92.050003   92.849998  89.300003   90.349998   50887668.0   
2022-05-10   83.500000   87.199997  83.150002   86.000000   64139439.0   
2022-05-11   83.699997   87.949997  83.599998   85.650002   63211009.0   
2022-05-12   81.849998   82.550003  79.900002   80.000000   69966835.0   
2022-05-13   82.199997   82.849998  81.250000   82.199997   53688518.0   

symbol        AZN.L                                        
               open     high      low    close     volume  
2022-05-03  10498.0  10710.0  10472.0  10684.0  2234172.0  
2022-05-04  10792.0  10792.0  10458.0  10506.0  2213731.0  
2022-05-05  10532.0  10632.0  10478.0  10512.0  2501672.0  
2022-05-06  10464.0  10474.0  10222.0  10322.0  2591818.0  
2022-05-10  10166.0  10338.0  10014.0  10246.0  1759026.0  
2022-05-11  10210.0  10262.0   9901.0  10026.0  4552005.0  
2022-05-12   9948.0   9984.0   9764.0   9965.0  3585647.0  
2022-05-13  10054.0  10300.0   9996.0  10286.0  2084381.0

`pt.get_trading_index` can be called to just return the trading index over the period covered by the table.

In [29]:
df_intraday.pt.get_trading_index(xhkg)

IntervalIndex([[2022-05-12 01:30:00, 2022-05-12 01:35:00), [2022-05-12 01:35:00, 2022-05-12 01:40:00), [2022-05-12 01:40:00, 2022-05-12 01:45:00), [2022-05-12 01:45:00, 2022-05-12 01:50:00), [2022-05-12 01:50:00, 2022-05-12 01:55:00) ... [2022-05-13 07:35:00, 2022-05-13 07:40:00), [2022-05-13 07:40:00, 2022-05-13 07:45:00), [2022-05-13 07:45:00, 2022-05-13 07:50:00), [2022-05-13 07:50:00, 2022-05-13 07:55:00), [2022-05-13 07:55:00, 2022-05-13 08:00:00)], dtype='interval[datetime64[ns, UTC], left]', length=132)

Recalling that a `NotImplementedError` is raised whenever a common property or method is not available to a specific PT class...

In [ ]:
df_comp.pt.reindex_to_calendar(xnys)

```
---------------------------------------------------------------------------
NotImplementedError                       Traceback (most recent call last)
<ipython-input-147-15a53ada8038> in <module>
----> 1 df_comp.pt.reindex_to_calendar(xnys)

NotImplementedError: reindex_to_calendar is not implemented for <class 'market_prices.pt.PTDailyIntradayComposite'>.
```

## Downsampling

`.pt.downsample` is available to `PT.Intraday` and `PT.Daily` to downsample the table to a higher interval (i.e. 'lower' frequency).

### `PTIntrday.downsample`

By default, an intraday table is downsampled on the basis of anchoring "workback". Indices are evaluated by working back from the last indice of the base table and each downsampled indice defined to comprise the number of _trading minutes_ represented by the downsampled interval (see the [anchors](#./anchor.ipynb) tutorial for further explanation of anchoring data).

In [31]:
df_intraday.pt.downsample("4H")

symbol                                            MSFT              \
                                                  open        high   
[2022-05-11 15:15:00, 2022-05-12 01:45:00)  261.445007  261.950012   
[2022-05-12 01:45:00, 2022-05-12 05:45:00)         NaN         NaN   
[2022-05-12 05:45:00, 2022-05-12 09:45:00)  257.690002  258.510010   
[2022-05-12 09:45:00, 2022-05-12 13:45:00)  255.639999  259.880005   
[2022-05-12 13:45:00, 2022-05-12 23:15:00)  251.610001  255.779999   
[2022-05-12 23:15:00, 2022-05-13 04:15:00)         NaN         NaN   
[2022-05-13 04:15:00, 2022-05-13 08:15:00)         NaN         NaN   
[2022-05-13 08:15:00, 2022-05-13 12:15:00)  258.250000  262.369995   

symbol                                                              \
                                                   low       close   
[2022-05-11 15:15:00, 2022-05-12 01:45:00)  259.299988  260.660004   
[2022-05-12 01:45:00, 2022-05-12 05:45:00)         NaN         NaN   
[2022-05-12 05:45:00, 2022-05-12 09:45:00)  252.979996  255.649994   
[2022-05-12 09:45:00, 2022-05-12 13:45:00)  251.619995  251.630005   
[2022-05-12 13:45:00, 2022-05-12 23:15:00)  250.020004  255.369995   
[2022-05-12 23:15:00, 2022-05-13 04:15:00)         NaN         NaN   
[2022-05-13 04:15:00, 2022-05-13 08:15:00)         NaN         NaN   
[2022-05-13 08:15:00, 2022-05-13 12:15:00)  255.350006  262.359985   

symbol                                                    9988.HK             \
                                                volume       open       high   
[2022-05-11 15:15:00, 2022-05-12 01:45:00)   7916061.0  81.849998  82.550003   
[2022-05-12 01:45:00, 2022-05-12 05:45:00)         NaN  80.349998  80.900002   
[2022-05-12 05:45:00, 2022-05-12 09:45:00)   5364984.0        NaN        NaN   
[2022-05-12 09:45:00, 2022-05-12 13:45:00)  23160001.0        NaN        NaN   
[2022-05-12 13:45:00, 2022-05-12 23:15:00)  16438821.0  82.199997  82.849998   
[2022-05-12 23:15:00, 2022-05-13 04:15:00)         NaN  82.000000  82.599998   
[2022-05-13 04:15:00, 2022-05-13 08:15:00)         NaN        NaN        NaN   
[2022-05-13 08:15:00, 2022-05-13 12:15:00)  15533175.0        NaN        NaN   

symbol                                                                        \
                                                  low      close      volume   
[2022-05-11 15:15:00, 2022-05-12 01:45:00)  80.199997  80.349998  30307080.0   
[2022-05-12 01:45:00, 2022-05-12 05:45:00)  79.800003  79.949997  24571676.0   
[2022-05-12 05:45:00, 2022-05-12 09:45:00)        NaN        NaN         NaN   
[2022-05-12 09:45:00, 2022-05-12 13:45:00)        NaN        NaN         NaN   
[2022-05-12 13:45:00, 2022-05-12 23:15:00)  81.250000  81.949997  15786284.0   
[2022-05-12 23:15:00, 2022-05-13 04:15:00)  80.000000  82.199997  27060178.0   
[2022-05-13 04:15:00, 2022-05-13 08:15:00)        NaN        NaN         NaN   
[2022-05-13 08:15:00, 2022-05-13 12:15:00)        NaN        NaN         NaN   

symbol                                             AZN.L           \
                                                    open     high   
[2022-05-11 15:15:00, 2022-05-12 01:45:00)           NaN      NaN   
[2022-05-12 01:45:00, 2022-05-12 05:45:00)   9948.000000   9950.0   
[2022-05-12 05:45:00, 2022-05-12 09:45:00)   9834.000000   9948.0   
[2022-05-12 09:45:00, 2022-05-12 13:45:00)   9880.000000   9984.0   
[2022-05-12 13:45:00, 2022-05-12 23:15:00)           NaN      NaN   
[2022-05-12 23:15:00, 2022-05-13 04:15:00)  10054.000000  10186.0   
[2022-05-13 04:15:00, 2022-05-13 08:15:00)  10167.620117  10280.0   
[2022-05-13 08:15:00, 2022-05-13 12:15:00)  10242.000000  10300.0   

symbol                                                                  \
                                                     low         close   
[2022-05-11 15:15:00, 2022-05-12 01:45:00)           NaN           NaN   
[2022-05-12 01:45:00, 2022-05-12 05:45:00)   9764.000000   9833.099609   
[2022-05-

Alternatively the downsampled data can be anchored on each session "open". In this case it's necessary to pass a `calendar` to provide knowledge of session times.

In [32]:
df_intraday.pt.downsample("4H", anchor="open", calendar=xnys)

symbol                                            MSFT              \
                                                  open        high   
[2022-05-11 13:30:00, 2022-05-11 17:30:00)  263.277191  264.427887   
[2022-05-11 17:30:00, 2022-05-11 21:30:00)         NaN         NaN   
[2022-05-11 21:30:00, 2022-05-12 01:30:00)         NaN         NaN   
[2022-05-12 01:30:00, 2022-05-12 05:30:00)         NaN         NaN   
[2022-05-12 05:30:00, 2022-05-12 09:30:00)         NaN         NaN   
[2022-05-12 09:30:00, 2022-05-12 13:30:00)  257.690002  259.880005   
[2022-05-12 13:30:00, 2022-05-12 17:30:00)  253.097900  255.779999   
[2022-05-12 17:30:00, 2022-05-12 21:30:00)         NaN         NaN   
[2022-05-12 21:30:00, 2022-05-13 01:30:00)         NaN         NaN   
[2022-05-13 01:30:00, 2022-05-13 05:30:00)         NaN         NaN   
[2022-05-13 05:30:00, 2022-05-13 09:30:00)         NaN         NaN   
[2022-05-13 09:30:00, 2022-05-13 13:30:00)  258.250000  262.369995   

symbol                                                              \
                                                   low       close   
[2022-05-11 13:30:00, 2022-05-11 17:30:00)  259.299988  260.660004   
[2022-05-11 17:30:00, 2022-05-11 21:30:00)         NaN         NaN   
[2022-05-11 21:30:00, 2022-05-12 01:30:00)         NaN         NaN   
[2022-05-12 01:30:00, 2022-05-12 05:30:00)         NaN         NaN   
[2022-05-12 05:30:00, 2022-05-12 09:30:00)         NaN         NaN   
[2022-05-12 09:30:00, 2022-05-12 13:30:00)  252.389999  253.100006   
[2022-05-12 13:30:00, 2022-05-12 17:30:00)  250.020004  255.369995   
[2022-05-12 17:30:00, 2022-05-12 21:30:00)         NaN         NaN   
[2022-05-12 21:30:00, 2022-05-13 01:30:00)         NaN         NaN   
[2022-05-13 01:30:00, 2022-05-13 05:30:00)         NaN         NaN   
[2022-05-13 05:30:00, 2022-05-13 09:30:00)         NaN         NaN   
[2022-05-13 09:30:00, 2022-05-13 13:30:00)  255.350006  262.359985   

symbol                                                    9988.HK             \
                                                volume       open       high   
[2022-05-11 13:30:00, 2022-05-11 17:30:00)  16421843.0        NaN        NaN   
[2022-05-11 17:30:00, 2022-05-11 21:30:00)         NaN        NaN        NaN   
[2022-05-11 21:30:00, 2022-05-12 01:30:00)         NaN  81.849998  82.550003   
[2022-05-12 01:30:00, 2022-05-12 05:30:00)         NaN  80.849998  80.949997   
[2022-05-12 05:30:00, 2022-05-12 09:30:00)         NaN        NaN        NaN   
[2022-05-12 09:30:00, 2022-05-12 13:30:00)  27567859.0        NaN        NaN   
[2022-05-12 13:30:00, 2022-05-12 17:30:00)  17395947.0        NaN        NaN   
[2022-05-12 17:30:00, 2022-05-12 21:30:00)         NaN        NaN        NaN   
[2022-05-12 21:30:00, 2022-05-13 01:30:00)         NaN  82.199997  82.849998   
[2022-05-13 01:30:00, 2022-05-13 05:30:00)         NaN  81.400002  82.599998   
[2022-05-13 05:30:00, 2022-05-13 09:30:00)         NaN        NaN        NaN   
[2022-05-13 09:30:00, 2022-05-13 13:30:00)  15533175.0        NaN        NaN   

symbol                                                                        \
                                                  low      close      volume   
[2022-05-11 13:30:00, 2022-05-11 17:30:00)        NaN        NaN         NaN   
[2022-05-11 17:30:00, 2022-05-11 21:30:00)        NaN        NaN         NaN   
[2022-05-11 21:30:00, 2022-05-12 01:30:00)  80.750000  80.849998  28015266.0   
[2022-05-12 01:30:00, 2022-05-12 05:30:00)  79.800003  79.949997  26863490.0   
[2022-05-12 05:30:00, 2022-05-12 09:30:00)        NaN        NaN         NaN   
[2022-05-12 09:30:00, 2022-05-12 13:30:00)        NaN        NaN         NaN   
[2022-05-12 13:30:00, 2022-05-12 17:30:00)        NaN        NaN         NaN   
[2022-05-12 17:30:00, 2022-05-12 21:30:00)        NaN        NaN         NaN   
[2022-05-12 21:30:00, 2022-05-13 01:30:00)  81.250000  81.349998  22618789.0   
[2022-05-13 01:30:00, 2022-05-13 05:3

Notice that there are rows with data missing for all symbols. When the symbols are not all associated with the same calendar (as is the case here) the downsampled data can be made more representative by additionally passing a `composite_calendar` that represents all the table's symbols. The composite calendar for prices returned by the `get` method is exposed by the Prices instance's `.composite_calendar` attribute, or the `.cc` alias.  

The additional information of each calendar's session bounds allows for the method to remove indices that do not include a trading period of any calendar.

In [33]:
df_ds = df_intraday.pt.downsample(
    "4H", anchor="open", calendar=xnys, composite_calendar=prices.cc
)
df_ds

symbol                                            MSFT              \
                                                  open        high   
[2022-05-11 13:30:00, 2022-05-11 17:30:00)  263.277191  264.427887   
[2022-05-11 21:30:00, 2022-05-12 01:30:00)         NaN         NaN   
[2022-05-12 01:30:00, 2022-05-12 05:30:00)         NaN         NaN   
[2022-05-12 05:30:00, 2022-05-12 09:30:00)         NaN         NaN   
[2022-05-12 09:30:00, 2022-05-12 13:30:00)  257.690002  259.880005   
[2022-05-12 13:30:00, 2022-05-12 17:30:00)  253.097900  255.779999   
[2022-05-12 21:30:00, 2022-05-13 01:30:00)         NaN         NaN   
[2022-05-13 01:30:00, 2022-05-13 05:30:00)         NaN         NaN   
[2022-05-13 05:30:00, 2022-05-13 09:30:00)         NaN         NaN   
[2022-05-13 09:30:00, 2022-05-13 13:30:00)  258.250000  262.369995   

symbol                                                              \
                                                   low       close   
[2022-05-11 13:30:00, 2022-05-11 17:30:00)  259.299988  260.660004   
[2022-05-11 21:30:00, 2022-05-12 01:30:00)         NaN         NaN   
[2022-05-12 01:30:00, 2022-05-12 05:30:00)         NaN         NaN   
[2022-05-12 05:30:00, 2022-05-12 09:30:00)         NaN         NaN   
[2022-05-12 09:30:00, 2022-05-12 13:30:00)  252.389999  253.100006   
[2022-05-12 13:30:00, 2022-05-12 17:30:00)  250.020004  255.369995   
[2022-05-12 21:30:00, 2022-05-13 01:30:00)         NaN         NaN   
[2022-05-13 01:30:00, 2022-05-13 05:30:00)         NaN         NaN   
[2022-05-13 05:30:00, 2022-05-13 09:30:00)         NaN         NaN   
[2022-05-13 09:30:00, 2022-05-13 13:30:00)  255.350006  262.359985   

symbol                                                    9988.HK             \
                                                volume       open       high   
[2022-05-11 13:30:00, 2022-05-11 17:30:00)  16421843.0        NaN        NaN   
[2022-05-11 21:30:00, 2022-05-12 01:30:00)         NaN  81.849998  82.550003   
[2022-05-12 01:30:00, 2022-05-12 05:30:00)         NaN  80.849998  80.949997   
[2022-05-12 05:30:00, 2022-05-12 09:30:00)         NaN        NaN        NaN   
[2022-05-12 09:30:00, 2022-05-12 13:30:00)  27567859.0        NaN        NaN   
[2022-05-12 13:30:00, 2022-05-12 17:30:00)  17395947.0        NaN        NaN   
[2022-05-12 21:30:00, 2022-05-13 01:30:00)         NaN  82.199997  82.849998   
[2022-05-13 01:30:00, 2022-05-13 05:30:00)         NaN  81.400002  82.599998   
[2022-05-13 05:30:00, 2022-05-13 09:30:00)         NaN        NaN        NaN   
[2022-05-13 09:30:00, 2022-05-13 13:30:00)  15533175.0        NaN        NaN   

symbol                                                                        \
                                                  low      close      volume   
[2022-05-11 13:30:00, 2022-05-11 17:30:00)        NaN        NaN         NaN   
[2022-05-11 21:30:00, 2022-05-12 01:30:00)  80.750000  80.849998  28015266.0   
[2022-05-12 01:30:00, 2022-05-12 05:30:00)  79.800003  79.949997  26863490.0   
[2022-05-12 05:30:00, 2022-05-12 09:30:00)        NaN        NaN         NaN   
[2022-05-12 09:30:00, 2022-05-12 13:30:00)        NaN        NaN         NaN   
[2022-05-12 13:30:00, 2022-05-12 17:30:00)        NaN        NaN         NaN   
[2022-05-12 21:30:00, 2022-05-13 01:30:00)  81.250000  81.349998  22618789.0   
[2022-05-13 01:30:00, 2022-05-13 05:30:00)  80.000000  82.199997  20227673.0   
[2022-05-13 05:30:00, 2022-05-13 09:30:00)        NaN        NaN         NaN   
[2022-05-13 09:30:00, 2022-05-13 13:30:00)        NaN        NaN         NaN   

symbol                                        AZN.L                    \
                                               open     high      low   
[2022-05-11 13:30:00, 2022-05-11 17:30:00)      NaN      NaN      NaN   
[2022-05-11 21:30:00, 2022-05-12 01:30:00)      NaN      NaN      NaN   
[2022-05-12 01:30:00, 2022-05-12 05:30:00)   9948.0   9950.0   9764.0   
[2022-05-12 05:30:00, 2022-05-1

The `curtail_end` option (default False) will remove the last indice if the base data was insufficient to represent the full period indicated, i.e. if the following is True then `curtail_end` will remove the final indice.

In [34]:
df_intraday.pt.last_ts < df_ds.pt.last_ts

True

In [35]:
df_intraday.pt.downsample(
    "4H",
    anchor="open",
    calendar=xnys,
    composite_calendar=prices.cc,
    curtail_end=True,
)[-2:]  # show only last two rows

symbol                                     MSFT                        \
                                           open high low close volume   
[2022-05-13 01:30:00, 2022-05-13 05:30:00)  NaN  NaN NaN   NaN    NaN   
[2022-05-13 05:30:00, 2022-05-13 09:30:00)  NaN  NaN NaN   NaN    NaN   

symbol                                        9988.HK                   \
                                                 open       high   low   
[2022-05-13 01:30:00, 2022-05-13 05:30:00)  81.400002  82.599998  80.0   
[2022-05-13 05:30:00, 2022-05-13 09:30:00)        NaN        NaN   NaN   

symbol                                                               AZN.L  \
                                                close      volume     open   
[2022-05-13 01:30:00, 2022-05-13 05:30:00)  82.199997  20227673.0  10054.0   
[2022-05-13 05:30:00, 2022-05-13 09:30:00)        NaN         NaN  10252.0   

symbol                                                                 \
                                               high      low    close   
[2022-05-13 01:30:00, 2022-05-13 05:30:00)  10272.0   9996.0  10256.0   
[2022-05-13 05:30:00, 2022-05-13 09:30:00)  10300.0  10186.0  10204.0   

symbol                                                
                                              volume  
[2022-05-13 01:30:00, 2022-05-13 05:30:00)  270642.0  
[2022-05-13 05:30:00, 2022-05-13 09:30:00)  998831.0

### `PTDaily.downsample`

#### Downsample to a **multiple of days**

By default, how a downsample frequency with unit "d" is interpreted depends on the table's frequency:
* If the table's frequency is a pandas `CustomBusinessDay` then the data will be downsampled in terms of the sessions that the table's frequency represents.
* Otherwise the data is downsampled according to days of the Gregorian calendar.

For example, the example daily table has no frequency (as it includes symbols associated with different calendars).

In [36]:
df_daily.pt.freq == None

False

Therefore downsampling to a multiple of days will by default downsample the table according to the days of the Gregorian calendar, i.e. each indice of the following indices will comprise five Gregorian calendar days.

In [37]:
df_daily.pt.downsample("5D")

symbol                          MSFT                                      \
                                open        high         low       close   
[2022-05-04, 2022-05-09)  282.589996  290.880005  271.269989  274.730011   
[2022-05-09, 2022-05-14)  270.059998  273.750000  250.020004  262.309998   

symbol                                9988.HK                         \
                               volume    open        high        low   
[2022-05-04, 2022-05-09)  114608000.0    98.5  101.099998  89.300003   
[2022-05-09, 2022-05-14)  202653130.0    83.5   87.949997  79.900002   

symbol                                              AZN.L               \
                              close       volume     open         high   
[2022-05-04, 2022-05-09)  90.349998  108056118.0  10792.0  10792.00000   
[2022-05-09, 2022-05-14)  82.199997  251005801.0  10372.0  10462.47168   

symbol                                                  
                              low    close      volume  
[2022-05-04, 2022-05-09)  10222.0  10322.0   7307221.0  
[2022-05-09, 2022-05-14)   9764.0  10286.0  14197984.0

However, the following alternative base table does have a frequency (as all symbols are associated with the same calendar). By default the downsampled indices are therefore evaluated such that they each comprise five sessions, rather than five calendar days.

In [38]:
prices_us = PricesYahoo("MSFT")
df_daily_alt = prices_us.get(days=10)
df_daily_alt.pt.freq

<CustomBusinessDay>

In [39]:
df_daily_alt.pt.freq == xnys.day

True

In [40]:
df_daily_alt.pt.downsample("5D", calendar=xnys)

symbol                          MSFT                                      \
                                open        high         low       close   
[2022-05-02, 2022-05-09)  277.709991  290.880005  271.269989  274.730011   
[2022-05-09, 2022-05-16)  270.059998  273.750000  250.020004  262.299988   

symbol                                 
                               volume  
[2022-05-02, 2022-05-09)  175737700.0  
[2022-05-09, 2022-05-16)  202666822.0

Notice that although the default treatment is to downsample based on sessions (if the table has a frequency), it's necessary to pass a calendar which has `.day` as the `CustomBusinessDay` being downsampled to. Whenever downsampling to a `CustomBusinessDay` (by default or otherwise) the method requires konwledge of the associated calendar and an error will be raised if `calendar` is not passed.

If the table has a `CustomBusinessDay` frequency although the downsampled data is required in terms of the Gregorian calendar, simply lose the frequency information from the table's index...

In [41]:
df_daily_alt.index.freq = None
df_daily_alt.pt.downsample("5D")

symbol                          MSFT                                      \
                                open        high         low       close   
[2022-05-04, 2022-05-09)  282.589996  290.880005  271.269989  274.730011   
[2022-05-09, 2022-05-14)  270.059998  273.750000  250.020004  262.299988   

symbol                                 
                               volume  
[2022-05-04, 2022-05-09)  114608000.0  
[2022-05-09, 2022-05-14)  202666822.0

Conversely, if wish to downsample a table without a frequency according to sessions of a particular calendar, pass `pdfreq` as a multiple of the CustomBusinessDay and `calendar` as the corresponding calendar.

In [42]:
pdfreq = xnys.day * 5
df_daily.pt.downsample(pdfreq, calendar=xnys)

symbol                          MSFT                                      \
                                open        high         low       close   
[2022-05-02, 2022-05-09)  277.709991  290.880005  271.269989  274.730011   
[2022-05-09, 2022-05-16)  270.059998  273.750000  250.020004  262.309998   

symbol                                9988.HK                         \
                               volume    open        high        low   
[2022-05-02, 2022-05-09)  175737700.0   100.0  103.099998  89.300003   
[2022-05-09, 2022-05-16)  202653130.0    83.5   87.949997  79.900002   

symbol                                              AZN.L               \
                              close       volume     open         high   
[2022-05-02, 2022-05-09)  90.349998  211294988.0  10498.0  10792.00000   
[2022-05-09, 2022-05-16)  82.199997  251005801.0  10372.0  10462.47168   

symbol                                                  
                              low    close      volume  
[2022-05-02, 2022-05-09)  10222.0  10322.0   9541393.0  
[2022-05-09, 2022-05-16)   9764.0  10286.0  14197984.0

#### Downsample to **units longer than daily**

Daily data can also be downsampled to monthly or quarterly intervals. These are always evaluated based on the Gregorian calendar.

Consider the following daily base table which covers 2021 and creeps into 2022.

In [43]:
df_daily_2021 = prices.get("1D", "2021", "2022-01-05")
df_daily_2021

symbol            MSFT                                                  \
                  open        high         low       close      volume   
2021-01-04  222.529999  223.000000  214.809998  217.690002  37130100.0   
2021-01-05  217.259995  218.520004  215.699997  217.899994  23823000.0   
2021-01-06  212.169998  216.490005  211.940002  212.250000  35930700.0   
2021-01-07  214.039993  219.339996  213.710007  218.289993  27694500.0   
2021-01-08  218.679993  220.580002  217.029999  219.619995  22956200.0   
...                ...         ...         ...         ...         ...   
2021-12-30  341.910004  343.130005  338.820007  339.320007  15994500.0   
2021-12-31  338.510010  339.359985  335.850006  336.320007  18000800.0   
2022-01-03  335.350006  338.000000  329.779999  334.750000  28865100.0   
2022-01-04  334.829987  335.200012  326.119995  329.010010  32674300.0   
2022-01-05  325.859985  326.070007  315.980011  316.380005  40054300.0   

symbol         9988.HK                                                  \
                  open        high         low       close      volume   
2021-01-04  224.800003  228.600006  223.399994  227.600006  27907724.0   
2021-01-05  220.000000  224.199997  219.199997  223.000000  35061260.0   
2021-01-06  228.000000  232.600006  227.800003  230.000000  31462508.0   
2021-01-07  221.000000  221.800003  216.399994  221.000000  47362360.0   
2021-01-08  223.399994  226.399994  219.600006  224.199997  29880895.0   
...                ...         ...         ...         ...         ...   
2021-12-30  110.000000  112.199997  109.199997  109.900002  23838394.0   
2021-12-31  119.500000  120.300003  118.099998  118.900002  31837667.0   
2022-01-03  117.000000  117.500000  114.000000  115.000000  22176946.0   
2022-01-04  118.400002  118.900002  115.699997  116.900002  23228903.0   
2022-01-05  119.000000  119.099998  113.900002  114.500000  30717509.0   

symbol       AZN.L                                         
              open        high     low   close     volume  
2021-01-04  7425.0  7560.00000  7402.5  7422.0  4373741.0  
2021-01-05  7445.0  7484.60791  7366.0  7439.0  2800020.0  
2021-01-06  7407.0  7583.00000  7375.0  7512.0  2567664.0  
2021-01-07  7575.0  7583.00000  7404.0  7475.0  2874639.0  
2021-01-08  7444.0  7499.00000  7398.0  7471.0  3028898.0  
...            ...         ...     ...     ...        ...  
2021-12-30  8649.0  8711.00000  8630.0  8673.0   697022.0  
2021-12-31  8689.0  8704.00000  8588.0  8678.0   482453.0  
2022-01-03     NaN         NaN     NaN     NaN        NaN  
2022-01-04  8626.0  8660.00000  8445.0  8473.0  2662591.0  
2022-01-05  8443.0  8524.00000  8426.0  8460.0  1775049.0  

[262 rows x 15 columns]

The data can be downsampled to a monthly interval.

In [44]:
df_daily_2021.pt.downsample("1MS")

symbol                          MSFT                                      \
                                open        high         low       close   
[2021-02-01, 2021-03-01)  235.059998  246.130005  227.880005  232.380005   
[2021-03-01, 2021-04-01)  235.899994  241.050003  224.259995  235.770004   
[2021-04-01, 2021-05-01)  238.470001  263.190002  238.050003  252.179993   
[2021-05-01, 2021-06-01)  253.399994  254.350006  238.070007  249.679993   
[2021-06-01, 2021-07-01)  251.229996  271.649994  243.000000  270.899994   
[2021-07-01, 2021-08-01)  269.609985  290.149994  269.600006  284.910004   
[2021-08-01, 2021-09-01)  286.359985  305.839996  283.739990  301.880005   
[2021-09-01, 2021-10-01)  302.869995  305.320007  281.619995  281.920013   
[2021-10-01, 2021-11-01)  282.119995  332.000000  280.250000  331.619995   
[2021-11-01, 2021-12-01)  331.359985  349.670013  326.369995  330.589996   
[2021-12-01, 2022-01-01)  335.130005  344.299988  317.250000  336.320007   

symbol                                    9988.HK                          \
                               volume        open        high         low   
[2021-02-01, 2021-03-01)  490962200.0  246.000000  269.399994  230.800003   
[2021-03-01, 2021-04-01)  724945800.0  234.800003  238.600006  214.000000   
[2021-04-01, 2021-05-01)  568661600.0  223.000000  241.600006  216.800003   
[2021-05-01, 2021-06-01)  495084900.0  225.000000  226.800003  200.199997   
[2021-06-01, 2021-07-01)  508572200.0  213.600006  225.000000  201.199997   
[2021-07-01, 2021-08-01)  522672700.0  216.199997  216.199997  174.500000   
[2021-08-01, 2021-09-01)  441308900.0  189.000000  197.600006  151.199997   
[2021-09-01, 2021-10-01)  502918700.0  163.500000  173.399994  141.000000   
[2021-10-01, 2021-11-01)  516515800.0  143.500000  179.899994  132.000000   
[2021-11-01, 2021-12-01)  509885200.0  161.100006  166.899994  123.599998   
[2021-12-01, 2022-01-01)  625674800.0  125.000000  127.000000  109.199997   

symbol                                              AZN.L               \
                               close       volume    open         high   
[2021-02-01, 2021-03-01)  232.399994  576784147.0  7485.0  7536.000000   
[2021-03-01, 2021-04-01)  220.000000  647654263.0  7025.0  7422.000000   
[2021-04-01, 2021-05-01)  225.000000  467653567.0  7269.0  8648.710938   
[2021-05-01, 2021-06-01)  211.000000  461010412.0  7828.0  8173.000000   
[2021-06-01, 2021-07-01)  220.000000  445826394.0  8095.0  8739.580078   
[2021-07-01, 2021-08-01)  189.000000  676613286.0  8700.0  8811.000000   
[2021-08-01, 2021-09-01)  165.500000  727602759.0  8271.0  8774.000000   
[2021-09-01, 2021-10-01)  142.199997  628931988.0  8535.0  8998.000000   
[2021-10-01, 2021-11-01)  163.000000  665825352.0  8881.0  9149.000000   
[2021-11-01, 2021-12-01)  127.300003  712797457.0  9177.0  9523.000000   
[2021-12-01, 2022-01-01)  118.900002  954133199.0  8303.0  8720.000000   

symbol                                                      
                                  low   close       volume  
[2021-02-01, 2021-03-01)  6895.000000  6945.0   55903671.0  
[2021-03-01, 2021-04-01)  6736.000000  7247.0   77291894.0  
[2021-04-01, 2021-05-01)  7045.000000  7715.0   52866142.0  
[2021-05-01, 2021-06-01)  6499.799805  8046.0   46613334.0  
[2021-06-01, 2021-07-01)  7870.000000  8683.0   52034874.0  
[2021-07-01, 2021-08-01)  8142.000000  8268.0  119203018.0  
[2021-08-01, 2021-09-01)  8105.000000  8514.0   51861902.0  
[2021-09-01, 2021-10-01)  8029.000000  8958.0   52332297.0  
[2021-10-01, 2021-11-01)  8607.000000  9094.0   31544334.0  
[2021-11-01, 2021-12-01)  8151.000000  8276.0   50473708.0  
[2021-12-01, 2022-01-01)  8090.319824  8678.0   40069581.0

As always, the intervals are closed on the "left" (the period covered by indice is inclusive of the indice's left bound but exclusive of the indice's right bound).

Notice that although the base data was requested from the start of 2021 the first indice of the downsampled table is for February, not January! This is because the first session of base data was 2021-01-04. The downsample method does not include January in order to not suggest that the indice includes data from the start of the year through 2021-01-04 - it has no way of knowing that this period did not include any sessions. The downsampling can be improved by passing `calendar` to give knowledge of dates that are not sessions. `calendar` can be passed as either a single calendar or a composite calendar.

In [45]:
df_daily_2021.pt.downsample("1MS", calendar=prices.cc)

symbol                          MSFT                                      \
                                open        high         low       close   
[2021-01-01, 2021-02-01)  222.529999  242.639999  211.940002  231.960007   
[2021-02-01, 2021-03-01)  235.059998  246.130005  227.880005  232.380005   
[2021-03-01, 2021-04-01)  235.899994  241.050003  224.259995  235.770004   
[2021-04-01, 2021-05-01)  238.470001  263.190002  238.050003  252.179993   
[2021-05-01, 2021-06-01)  253.399994  254.350006  238.070007  249.679993   
[2021-06-01, 2021-07-01)  251.229996  271.649994  243.000000  270.899994   
[2021-07-01, 2021-08-01)  269.609985  290.149994  269.600006  284.910004   
[2021-08-01, 2021-09-01)  286.359985  305.839996  283.739990  301.880005   
[2021-09-01, 2021-10-01)  302.869995  305.320007  281.619995  281.920013   
[2021-10-01, 2021-11-01)  282.119995  332.000000  280.250000  331.619995   
[2021-11-01, 2021-12-01)  331.359985  349.670013  326.369995  330.589996   
[2021-12-01, 2022-01-01)  335.130005  344.299988  317.250000  336.320007   

symbol                                    9988.HK                          \
                               volume        open        high         low   
[2021-01-01, 2021-02-01)  648076400.0  224.800003  270.000000  216.399994   
[2021-02-01, 2021-03-01)  490962200.0  246.000000  269.399994  230.800003   
[2021-03-01, 2021-04-01)  724945800.0  234.800003  238.600006  214.000000   
[2021-04-01, 2021-05-01)  568661600.0  223.000000  241.600006  216.800003   
[2021-05-01, 2021-06-01)  495084900.0  225.000000  226.800003  200.199997   
[2021-06-01, 2021-07-01)  508572200.0  213.600006  225.000000  201.199997   
[2021-07-01, 2021-08-01)  522672700.0  216.199997  216.199997  174.500000   
[2021-08-01, 2021-09-01)  441308900.0  189.000000  197.600006  151.199997   
[2021-09-01, 2021-10-01)  502918700.0  163.500000  173.399994  141.000000   
[2021-10-01, 2021-11-01)  516515800.0  143.500000  179.899994  132.000000   
[2021-11-01, 2021-12-01)  509885200.0  161.100006  166.899994  123.599998   
[2021-12-01, 2022-01-01)  625674800.0  125.000000  127.000000  109.199997   

symbol                                              AZN.L               \
                               close       volume    open         high   
[2021-01-01, 2021-02-01)  246.800003  842861890.0  7425.0  8014.000000   
[2021-02-01, 2021-03-01)  232.399994  576784147.0  7485.0  7536.000000   
[2021-03-01, 2021-04-01)  220.000000  647654263.0  7025.0  7422.000000   
[2021-04-01, 2021-05-01)  225.000000  467653567.0  7269.0  8648.710938   
[2021-05-01, 2021-06-01)  211.000000  461010412.0  7828.0  8173.000000   
[2021-06-01, 2021-07-01)  220.000000  445826394.0  8095.0  8739.580078   
[2021-07-01, 2021-08-01)  189.000000  676613286.0  8700.0  8811.000000   
[2021-08-01, 2021-09-01)  165.500000  727602759.0  8271.0  8774.000000   
[2021-09-01, 2021-10-01)  142.199997  628931988.0  8535.0  8998.000000   
[2021-10-01, 2021-11-01)  163.000000  665825352.0  8881.0  9149.000000   
[2021-11-01, 2021-12-01)  127.300003  712797457.0  9177.0  9523.000000   
[2021-12-01, 2022-01-01)  118.900002  954133199.0  8303.0  8720.000000   

symbol                                                      
                                  low   close       volume  
[2021-01-01, 2021-02-01)  7366.000000  7491.0   61660415.0  
[2021-02-01, 2021-03-01)  6895.000000  6945.0   55903671.0  
[2021-03-01, 2021-04-01)  6736.000000  7247.0   77291894.0  
[2021-04-01, 2021-05-01)  7045.000000  7715.0   52866142.0  
[2021-05-01, 2021-06-01)  6499.799805  8046.0   46613334.0  
[2021-06-01, 2021-07-01)  7870.000000  8683.0   52034874.0  
[2021-07-01, 2021-08-01)  8142.000000  8268.0  119203018.0  
[2021-08-01, 2021-09-01)  8105.000000  8514.0   51861902.0  
[2021-09-01, 2021-10-01)  8029.000000  8958.0   52332297.0  
[2021-10-01, 2021-11-01)  8607.000000  9094.0   31544334.0  
[2021-11-01, 2021-12-01)  8151.000000  8276.0   50473708.0  
[2021-12-01, 2022-01-01)  

The above table now includes January as the method inspects the composite calendar and can see that by including this indice it's not introducing a session for which base data is not available.

Also notice that the downsampled data does not include January 2022 even though the base data included the first few days of that month. Again, this is to not suggest prices cover a period they may not (from '2022-01-06' to the end of the month). But what if 'today' is '2022-01-05', such that it would be reasonable to include this indice? Pass `drop_incomplete_last_indice` as False (default True).

In [46]:
df_daily_2021.pt.downsample(
    "1MS", calendar=prices.cc, drop_incomplete_last_indice=False
)[-2:]

symbol                          MSFT                                      \
                                open        high         low       close   
[2021-12-01, 2022-01-01)  335.130005  344.299988  317.250000  336.320007   
[2022-01-01, 2022-02-01)  335.350006  338.000000  315.980011  316.380005   

symbol                                9988.HK                          \
                               volume    open        high         low   
[2021-12-01, 2022-01-01)  625674800.0   125.0  127.000000  109.199997   
[2022-01-01, 2022-02-01)  101593700.0   117.0  119.099998  113.900002   

symbol                                              AZN.L          \
                               close       volume    open    high   
[2021-12-01, 2022-01-01)  118.900002  954133199.0  8303.0  8720.0   
[2022-01-01, 2022-02-01)  114.500000   76123358.0  8626.0  8660.0   

symbol                                                     
                                  low   close      volume  
[2021-12-01, 2022-01-01)  8090.319824  8678.0  40069581.0  
[2022-01-01, 2022-02-01)  8426.000000  8460.0   4437640.0

Lastly, an example of downsampling daily data to a quarterly interval.

In [47]:
df_daily_2021.pt.downsample("QS", calendar=prices.cc)

symbol                          MSFT                                      \
                                open        high         low       close   
[2021-01-01, 2021-04-01)  222.529999  246.130005  211.940002  235.770004   
[2021-04-01, 2021-07-01)  238.470001  271.649994  238.050003  270.899994   
[2021-07-01, 2021-10-01)  269.609985  305.839996  269.600006  281.920013   
[2021-10-01, 2022-01-01)  282.119995  349.670013  280.250000  336.320007   

symbol                                     9988.HK                          \
                                volume        open        high         low   
[2021-01-01, 2021-04-01)  1.863984e+09  224.800003  270.000000  214.000000   
[2021-04-01, 2021-07-01)  1.572319e+09  223.000000  241.600006  200.199997   
[2021-07-01, 2021-10-01)  1.466900e+09  216.199997  216.199997  141.000000   
[2021-10-01, 2022-01-01)  1.652076e+09  143.500000  179.899994  109.199997   

symbol                                               AZN.L               \
                               close        volume    open         high   
[2021-01-01, 2021-04-01)  220.000000  2.067300e+09  7425.0  8014.000000   
[2021-04-01, 2021-07-01)  220.000000  1.374490e+09  7269.0  8739.580078   
[2021-07-01, 2021-10-01)  142.199997  2.033148e+09  8700.0  8998.000000   
[2021-10-01, 2022-01-01)  118.900002  2.332756e+09  8881.0  9523.000000   

symbol                                                      
                                  low   close       volume  
[2021-01-01, 2021-04-01)  6736.000000  7247.0  194855980.0  
[2021-04-01, 2021-07-01)  6499.799805  8683.0  151514350.0  
[2021-07-01, 2021-10-01)  8029.000000  8958.0  223397217.0  
[2021-10-01, 2022-01-01)  8090.319824  8678.0  122087623.0

...or a multiple of.

In [48]:
df_daily_2021.pt.downsample("2QS", calendar=prices.cc)

symbol                          MSFT                                      \
                                open        high         low       close   
[2021-01-01, 2021-07-01)  222.529999  271.649994  211.940002  270.899994   
[2021-07-01, 2022-01-01)  269.609985  349.670013  269.600006  336.320007   

symbol                                     9988.HK                          \
                                volume        open        high         low   
[2021-01-01, 2021-07-01)  3.436303e+09  224.800003  270.000000  200.199997   
[2021-07-01, 2022-01-01)  3.118976e+09  216.199997  216.199997  109.199997   

symbol                                               AZN.L               \
                               close        volume    open         high   
[2021-01-01, 2021-07-01)  220.000000  3.441791e+09  7425.0  8739.580078   
[2021-07-01, 2022-01-01)  118.900002  4.365904e+09  8700.0  9523.000000   

symbol                                                      
                                  low   close       volume  
[2021-01-01, 2021-07-01)  6499.799805  8683.0  346370330.0  
[2021-07-01, 2022-01-01)  8029.000000  8678.0  345484840.0

## Indices trading status

The following methods allow for querying if/which indices represent trading periods, non-trading periods or partial trading periods of a specific calendar. These methods are not further covered here - see the [.pt accessor method section of the anchor tutorial](./anchor.ipynb#.pt-accessor-methods) for comprehensive example usage.

* `.pt.indices_trading_status`
* `.pt.indices_trading`
* `.pt.indices_non_trading`
* `.pt.indices_partial_trading`
* `.pt.indices_partial_trading_info`
* `.pt.indices_all_trading`

## Querying prices

Various methods provide for extracting price data at a specific date or minute or  over a specified period.

`.pt.get_subset_from_indices` provides for simply getting a subset between specified indices. Note that the start and end of the requested subset must represent actual indice bounds.

In [49]:
start = df_intraday.index[-4].left  # to get a valid start minute
end = df_intraday.index[-2].right  # to get a valid end minute
print(f"{start=}\n{end=}\n")  # for reference

df_intraday.pt.get_subset_from_indices(start, end)

start=Timestamp('2022-05-13 11:55:00-0400', tz='America/New_York')
end=Timestamp('2022-05-13 12:10:00-0400', tz='America/New_York')



symbol                                            MSFT              \
                                                  open        high   
[2022-05-13 11:55:00, 2022-05-13 12:00:00)  261.839996  261.855011   
[2022-05-13 12:00:00, 2022-05-13 12:05:00)  261.339996  261.619995   
[2022-05-13 12:05:00, 2022-05-13 12:10:00)  261.589996  262.019989   

symbol                                                                        \
                                                   low       close    volume   
[2022-05-13 11:55:00, 2022-05-13 12:00:00)  261.149994  261.309998  202250.0   
[2022-05-13 12:00:00, 2022-05-13 12:05:00)  261.170013  261.600006  179609.0   
[2022-05-13 12:05:00, 2022-05-13 12:10:00)  261.369995  262.019989  288129.0   

symbol                                     9988.HK                        \
                                              open high low close volume   
[2022-05-13 11:55:00, 2022-05-13 12:00:00)     NaN  NaN NaN   NaN    NaN   
[2022-05-13 12:00:00, 2022-05-13 12:05:00)     NaN  NaN NaN   NaN    NaN   
[2022-05-13 12:05:00, 2022-05-13 12:10:00)     NaN  NaN NaN   NaN    NaN   

symbol                                     AZN.L                        
                                            open high low close volume  
[2022-05-13 11:55:00, 2022-05-13 12:00:00)   NaN  NaN NaN   NaN    NaN  
[2022-05-13 12:00:00, 2022-05-13 12:05:00)   NaN  NaN NaN   NaN    NaN  
[2022-05-13 12:05:00, 2022-05-13 12:10:00)   NaN  NaN NaN   NaN    NaN

In [50]:
start = df_daily.index[-4]  # to get a valid start session
end = df_daily.index[-2]  # to get a valid end session
print(f"{start=}\n{end=}\n")  # for reference

df_daily.pt.get_subset_from_indices(start, end)

start=Timestamp('2022-05-10 00:00:00', freq='B')
end=Timestamp('2022-05-12 00:00:00', freq='B')



symbol            MSFT                                                \
                  open        high         low       close    volume   
2022-05-10  271.690002  273.750000  265.070007  269.500000  39336400   
2022-05-11  265.679993  271.359985  259.299988  260.549988  48975900   
2022-05-12  257.690002  259.880005  250.020004  255.350006  50970700   

symbol        9988.HK                                                 AZN.L  \
                 open       high        low      close      volume     open   
2022-05-10  83.500000  87.199997  83.150002  86.000000  64139439.0  10166.0   
2022-05-11  83.699997  87.949997  83.599998  85.650002  63211009.0  10210.0   
2022-05-12  81.849998  82.550003  79.900002  80.000000  69966835.0   9948.0   

symbol                                            
               high      low    close     volume  
2022-05-10  10338.0  10014.0  10246.0  1759026.0  
2022-05-11  10262.0   9901.0  10026.0  4552005.0  
2022-05-12   9984.0   9764.0   9965.0  3585647.0

`.pt.price_at` is available to `PT.Intraday` to get prices as at a specific minute. The minute does not need to reflect an indice bound or lie within an indice, but rather simply be in the period covered by the table.

In [51]:
minute = df_intraday.index[-1].left
print(f"{minute=}\n") # for reference

df_intraday.pt.price_at(minute)

minute=Timestamp('2022-05-13 12:10:00-0400', tz='America/New_York')



symbol,MSFT,9988.HK,AZN.L
2022-05-13 12:10:00-04:00,262.024994,82.199997,10252.0


Notice that the price for "9988.HK" is present even though the Hong Kong exchange is not open on this minute; the prices represent the price 'as at' the minute, if a symbol's exchange is closed at that time then the price will be as at the prior close.

`.pt.price_at` is also available for the `PTDailyIntradayComposite` class if the passed minute falls within the period covered by the intraday part of the table.

`.pt.close_at` is available to `PTDaily` to get the price for each symbol as at the end of a given 'day'.

In [52]:
date = df_daily.index[-2]  # to get a date
print(f"{date}\n")  # for reference

df_daily.pt.close_at(date)

2022-05-12 00:00:00



symbol,MSFT,9988.HK,AZN.L
2022-05-12,255.350006,80.0,9965.0


`pt.close_at` can take any date in the period covered by the table, if a symbol is not open on the passed date then it's price will be defined as at the prior close. The method is also available for the `PTDailyIntradayComposite` class if the date falls within the period covered by the daily part of the table.

`.pt.session_prices` returns the prices for a specific session. Unlike `.pt.close_at`, `.pt.session_prices` requires an actual session to be passed, not merely a date. As `.pt.close_at`, it is available only to `PTDaily` class and the daily part of `PTDailyIntradayComposite`.

In [53]:
session = df_comp.pt.naive.index[1].left  # to get a valid session
print(f"{session=}\n")  # for reference

df_comp.pt.session_prices(session)

session=Timestamp('2020-05-13 00:00:00')



symbol            MSFT                                             9988.HK  \
                  open        high         low   close      volume    open   
2020-05-13  182.550003  184.050003  176.539993  179.75  44711500.0   195.5   

symbol                                                       AZN.L          \
                  high         low       close      volume    open    high   
2020-05-13  197.399994  194.300003  196.300003  26074457.0  8800.0  9056.0   

symbol                                 
               low   close     volume  
2020-05-13  8775.0  9004.0  2148708.0

## Tidying data

`.pt.data_for_all_start` and `.pt.data_for_all_end` return a copy of the table with indices with _any_ missing prices removed from the, respectively, start and end of the table.

`pt.data_for_all` removes such indices from both the start and end of the table.

In the intraday example at least one symbol is closed over every indice, such that all data is removed.

In [54]:
df_intraday.pt.data_for_all_start

Empty DataFrame
Columns: [(MSFT, open), (MSFT, high), (MSFT, low), (MSFT, close), (MSFT, volume), (9988.HK, open), (9988.HK, high), (9988.HK, low), (9988.HK, close), (9988.HK, volume), (AZN.L, open), (AZN.L, high), (AZN.L, low), (AZN.L, close), (AZN.L, volume)]
Index: []

The methods are more useful with daily data...

In [55]:
df_daily.pt.data_for_all

symbol            MSFT                                                \
                  open        high         low       close    volume   
2022-05-03  283.959991  284.130005  280.149994  281.779999  25978600   
2022-05-04  282.589996  290.880005  276.730011  289.980011  33599300   
2022-05-05  285.540009  286.350006  274.339996  277.350006  43260400   
2022-05-06  274.809998  279.250000  271.269989  274.730011  37748300   
2022-05-09  270.059998  272.359985  263.320007  264.579987  47726000   
2022-05-10  271.690002  273.750000  265.070007  269.500000  39336400   
2022-05-11  265.679993  271.359985  259.299988  260.549988  48975900   
2022-05-12  257.690002  259.880005  250.020004  255.350006  50970700   
2022-05-13  257.350006  262.369995  255.350006  262.309998  15644130   

symbol         9988.HK                                                  \
                  open        high        low       close       volume   
2022-05-03  100.000000  103.099998  92.500000  100.300003  103238870.0   
2022-05-04   98.500000   98.550003  95.849998   96.550003   29177225.0   
2022-05-05   99.500000  101.099998  96.199997   96.699997   27991225.0   
2022-05-06   92.050003   92.849998  89.300003   90.349998   50887668.0   
2022-05-09         NaN         NaN        NaN         NaN          NaN   
2022-05-10   83.500000   87.199997  83.150002   86.000000   64139439.0   
2022-05-11   83.699997   87.949997  83.599998   85.650002   63211009.0   
2022-05-12   81.849998   82.550003  79.900002   80.000000   69966835.0   
2022-05-13   82.199997   82.849998  81.250000   82.199997   53688518.0   

symbol        AZN.L                                            
               open         high      low    close     volume  
2022-05-03  10498.0  10710.00000  10472.0  10684.0  2234172.0  
2022-05-04  10792.0  10792.00000  10458.0  10506.0  2213731.0  
2022-05-05  10532.0  10632.00000  10478.0  10512.0  2501672.0  
2022-05-06  10464.0  10474.00000  10222.0  10322.0  2591818.0  
2022-05-09  10372.0  10462.47168  10118.0  10140.0  2216925.0  
2022-05-10  10166.0  10338.00000  10014.0  10246.0  1759026.0  
2022-05-11  10210.0  10262.00000   9901.0  10026.0  4552005.0  
2022-05-12   9948.0   9984.00000   9764.0   9965.0  3585647.0  
2022-05-13  10054.0  10300.00000   9996.0  10286.0  2084381.0

...or with symbols trading on exchanges that overlap

In [56]:
prices_alt = PricesYahoo("MSFT, BTC-USD")
end = df_daily.index[-2]
print(f"{end=}\n")  # for reference

df = prices_alt.get("8H", end=end, days=2, lead_symbol="BTC-USD")
df

end=Timestamp('2022-05-12 00:00:00', freq='B')



symbol                                            MSFT              \
                                                  open        high   
[2022-05-11, 2022-05-11 08:00:00)                  NaN         NaN   
[2022-05-11 08:00:00, 2022-05-11 16:00:00)  265.679993  271.070007   
[2022-05-11 16:00:00, 2022-05-12)           265.420013  266.299988   
[2022-05-12, 2022-05-12 08:00:00)                  NaN         NaN   
[2022-05-12 08:00:00, 2022-05-12 16:00:00)  257.690002  259.880005   
[2022-05-12 16:00:00, 2022-05-13)           254.809998  256.410004   

symbol                                                              \
                                                   low       close   
[2022-05-11, 2022-05-11 08:00:00)                  NaN         NaN   
[2022-05-11 08:00:00, 2022-05-11 16:00:00)  263.790314  265.450012   
[2022-05-11 16:00:00, 2022-05-12)           259.299988  260.660004   
[2022-05-12, 2022-05-12 08:00:00)                  NaN         NaN   
[2022-05-12 08:00:00, 2022-05-12 16:00:00)  252.979996  254.830002   
[2022-05-12 16:00:00, 2022-05-13)           250.020004  255.369995   

symbol                                                       BTC-USD  \
                                                volume          open   
[2022-05-11, 2022-05-11 08:00:00)                  NaN  31020.472656   
[2022-05-11 08:00:00, 2022-05-11 16:00:00)  15647969.0  30455.015625   
[2022-05-11 16:00:00, 2022-05-12)           22206076.0  30951.101562   
[2022-05-12, 2022-05-12 08:00:00)                  NaN  28936.734375   
[2022-05-12 08:00:00, 2022-05-12 16:00:00)  20171708.0  27938.609375   
[2022-05-12 16:00:00, 2022-05-13)           24792098.0  29067.603516   

symbol                                                                  \
                                                    high           low   
[2022-05-11, 2022-05-11 08:00:00)           31732.447266  30330.562500   
[2022-05-11 08:00:00, 2022-05-11 16:00:00)  31988.117188  29323.230469   
[2022-05-11 16:00:00, 2022-05-12)           31068.644531  28170.412109   
[2022-05-12, 2022-05-12 08:00:00)           29989.203125  26350.490234   
[2022-05-12 08:00:00, 2022-05-12 16:00:00)  29649.582031  27352.449219   
[2022-05-12 16:00:00, 2022-05-13)           29761.673828  28031.779297   

symbol                                                                  
                                                   close        volume  
[2022-05-11, 2022-05-11 08:00:00)           30446.466797  4.552090e+09  
[2022-05-11 08:00:00, 2022-05-11 16:00:00)  30916.451172  2.288354e+10  
[2022-05-11 16:00:00, 2022-05-12)           28936.253906  1.048805e+10  
[2022-05-12, 2022-05-12 08:00:00)           27966.919922  1.987425e+10  
[2022-05-12 08:00:00, 2022-05-12 16:00:00)  29085.171875  1.256123e+10  
[2022-05-12 16:00:00, 2022-05-13)           29063.316406  5.362573e+09

In [57]:
df.pt.data_for_all

symbol                                            MSFT              \
                                                  open        high   
[2022-05-11 08:00:00, 2022-05-11 16:00:00)  265.679993  271.070007   
[2022-05-11 16:00:00, 2022-05-12)           265.420013  266.299988   
[2022-05-12, 2022-05-12 08:00:00)                  NaN         NaN   
[2022-05-12 08:00:00, 2022-05-12 16:00:00)  257.690002  259.880005   
[2022-05-12 16:00:00, 2022-05-13)           254.809998  256.410004   

symbol                                                              \
                                                   low       close   
[2022-05-11 08:00:00, 2022-05-11 16:00:00)  263.790314  265.450012   
[2022-05-11 16:00:00, 2022-05-12)           259.299988  260.660004   
[2022-05-12, 2022-05-12 08:00:00)                  NaN         NaN   
[2022-05-12 08:00:00, 2022-05-12 16:00:00)  252.979996  254.830002   
[2022-05-12 16:00:00, 2022-05-13)           250.020004  255.369995   

symbol                                                       BTC-USD  \
                                                volume          open   
[2022-05-11 08:00:00, 2022-05-11 16:00:00)  15647969.0  30455.015625   
[2022-05-11 16:00:00, 2022-05-12)           22206076.0  30951.101562   
[2022-05-12, 2022-05-12 08:00:00)                  NaN  28936.734375   
[2022-05-12 08:00:00, 2022-05-12 16:00:00)  20171708.0  27938.609375   
[2022-05-12 16:00:00, 2022-05-13)           24792098.0  29067.603516   

symbol                                                                  \
                                                    high           low   
[2022-05-11 08:00:00, 2022-05-11 16:00:00)  31988.117188  29323.230469   
[2022-05-11 16:00:00, 2022-05-12)           31068.644531  28170.412109   
[2022-05-12, 2022-05-12 08:00:00)           29989.203125  26350.490234   
[2022-05-12 08:00:00, 2022-05-12 16:00:00)  29649.582031  27352.449219   
[2022-05-12 16:00:00, 2022-05-13)           29761.673828  28031.779297   

symbol                                                                  
                                                   close        volume  
[2022-05-11 08:00:00, 2022-05-11 16:00:00)  30916.451172  2.288354e+10  
[2022-05-11 16:00:00, 2022-05-12)           28936.253906  1.048805e+10  
[2022-05-12, 2022-05-12 08:00:00)           27966.919922  1.987425e+10  
[2022-05-12 08:00:00, 2022-05-12 16:00:00)  29085.171875  1.256123e+10  
[2022-05-12 16:00:00, 2022-05-13)           29063.316406  5.362573e+09

`.pt.fillna` will return a copy of the table with missing values filled either forward ("ffill"), backwards ("bfill") or 'forwards then backwards' to ensure all missing values are filled ("both").

In [58]:
df.pt.fillna("both")

symbol                                            MSFT              \
                                                  open        high   
[2022-05-11, 2022-05-11 08:00:00)           265.679993  265.679993   
[2022-05-11 08:00:00, 2022-05-11 16:00:00)  265.679993  271.070007   
[2022-05-11 16:00:00, 2022-05-12)           265.420013  266.299988   
[2022-05-12, 2022-05-12 08:00:00)           260.660004  260.660004   
[2022-05-12 08:00:00, 2022-05-12 16:00:00)  257.690002  259.880005   
[2022-05-12 16:00:00, 2022-05-13)           254.809998  256.410004   

symbol                                                              \
                                                   low       close   
[2022-05-11, 2022-05-11 08:00:00)           265.679993  265.679993   
[2022-05-11 08:00:00, 2022-05-11 16:00:00)  263.790314  265.450012   
[2022-05-11 16:00:00, 2022-05-12)           259.299988  260.660004   
[2022-05-12, 2022-05-12 08:00:00)           260.660004  260.660004   
[2022-05-12 08:00:00, 2022-05-12 16:00:00)  252.979996  254.830002   
[2022-05-12 16:00:00, 2022-05-13)           250.020004  255.369995   

symbol                                                       BTC-USD  \
                                                volume          open   
[2022-05-11, 2022-05-11 08:00:00)                  0.0  31020.472656   
[2022-05-11 08:00:00, 2022-05-11 16:00:00)  15647969.0  30455.015625   
[2022-05-11 16:00:00, 2022-05-12)           22206076.0  30951.101562   
[2022-05-12, 2022-05-12 08:00:00)                  0.0  28936.734375   
[2022-05-12 08:00:00, 2022-05-12 16:00:00)  20171708.0  27938.609375   
[2022-05-12 16:00:00, 2022-05-13)           24792098.0  29067.603516   

symbol                                                                  \
                                                    high           low   
[2022-05-11, 2022-05-11 08:00:00)           31732.447266  30330.562500   
[2022-05-11 08:00:00, 2022-05-11 16:00:00)  31988.117188  29323.230469   
[2022-05-11 16:00:00, 2022-05-12)           31068.644531  28170.412109   
[2022-05-12, 2022-05-12 08:00:00)           29989.203125  26350.490234   
[2022-05-12 08:00:00, 2022-05-12 16:00:00)  29649.582031  27352.449219   
[2022-05-12 16:00:00, 2022-05-13)           29761.673828  28031.779297   

symbol                                                                  
                                                   close        volume  
[2022-05-11, 2022-05-11 08:00:00)           30446.466797  4.552090e+09  
[2022-05-11 08:00:00, 2022-05-11 16:00:00)  30916.451172  2.288354e+10  
[2022-05-11 16:00:00, 2022-05-12)           28936.253906  1.048805e+10  
[2022-05-12, 2022-05-12 08:00:00)           27966.919922  1.987425e+10  
[2022-05-12 08:00:00, 2022-05-12 16:00:00)  29085.171875  1.256123e+10  
[2022-05-12 16:00:00, 2022-05-13)           29063.316406  5.362573e+09

Note how the price columns are filled:
* Price columns filled "ffill" take value as prior 'close'.
* Price columns filled "bfill" take value as subsequent 'open'.
* Filled 'volume' columns take 0.

The `.pt.stacked` property will return a copy of a single-row price table with the symbols stacked to separate rows.

In [59]:
# get a single-row price table
single_row_df = df.pt.data_for_all_start.iloc[[0]]
single_row_df

symbol                                            MSFT              \
                                                  open        high   
[2022-05-11 08:00:00, 2022-05-11 16:00:00)  265.679993  271.070007   

symbol                                                              \
                                                   low       close   
[2022-05-11 08:00:00, 2022-05-11 16:00:00)  263.790314  265.450012   

symbol                                                       BTC-USD  \
                                                volume          open   
[2022-05-11 08:00:00, 2022-05-11 16:00:00)  15647969.0  30455.015625   

symbol                                                                  \
                                                    high           low   
[2022-05-11 08:00:00, 2022-05-11 16:00:00)  31988.117188  29323.230469   

symbol                                                                  
                                                   close        volume  
[2022-05-11 08:00:00, 2022-05-11 16:00:00)  30916.451172  2.288354e+10

In [60]:
single_row_df.pt.stacked

open  \
                                           symbol                  
[2022-05-11 08:00:00, 2022-05-11 16:00:00) BTC-USD  30455.015625   
                                           MSFT       265.679993   

                                                            high  \
                                           symbol                  
[2022-05-11 08:00:00, 2022-05-11 16:00:00) BTC-USD  31988.117188   
                                           MSFT       271.070007   

                                                             low  \
                                           symbol                  
[2022-05-11 08:00:00, 2022-05-11 16:00:00) BTC-USD  29323.230469   
                                           MSFT       263.790314   

                                                           close        volume  
                                           symbol                               
[2022-05-11 08:00:00, 2022-05-11 16:00:00) BTC-USD  30916.451172  2.288354e+10  
                                           MSFT       265.450012  1.564797e+07

The stacked `DataFrame` will have five columns, 'open', 'low', 'high', 'close', 'volume', and one row for each symbol. The index becomes `pd.MultiIndex` with level 0 as the prior index.

Only single-row price tables can be stacked.

### `.operate`

The `.pt.operate` has a host of optional kwargs that collectively provide for undertaking multiple 'tidying' operations with a single call.

The following are convenience options that in turn call methods already covered above:
* **`tz`** to change the index's timezone. For `PTIntraday` any `pytz` timezone or `None` are valid values. For all other PT classes only `None` and "UTC" (or `pytz.UTC`) are valid values.
* **`fill`** can take "bfill", "ffill" or "both" and behaves as `.pt.fillna`.
* __`data_for_all*`__. If passed as `True` these options behave as the eponymous `.pt` methods covered above:
    * `data_for_all`
    * `data_for_all_start`
    * `data_for_all_end`

The following options have no `.pt` equivalent and can only be actioned via `.pt.operate`.

**`include`** returns the table for only the passed symbols whilst **`exclude`** returns the table with the passed symbols excluded (both options cannot be passed together).

In [61]:
df_daily.pt.operate(include="MSFT")[:2]

symbol            MSFT                                              
                  open        high         low       close    volume
2022-05-02  277.709991  284.940002  276.220001  284.470001  35151100
2022-05-03  283.959991  284.130005  280.149994  281.779999  25978600

In [62]:
df_intraday.pt.operate(exclude=["MSFT", "9988.HK"])[:2]

symbol                                     AZN.L                      
                                            open high low close volume
[2022-05-11 12:15:00, 2022-05-11 12:20:00)   NaN  NaN NaN   NaN    NaN
[2022-05-11 12:20:00, 2022-05-11 12:25:00)   NaN  NaN NaN   NaN    NaN

**`side`** will change an `IntervalIndex` to a `DatetimeIndex` based on either the "left" or "right" side of the `IntervalIndex` (this option is ignored by the `PTDaily` class).

In [63]:
df_intraday.pt.operate(side="left")[:2]

symbol                           MSFT                                      \
                                 open        high         low       close   
2022-05-11 12:15:00-04:00  265.250000  265.522614  264.700012  264.809998   
2022-05-11 12:20:00-04:00  264.790009  266.299988  264.709991  266.200012   

symbol                              9988.HK                       AZN.L       \
                             volume    open high low close volume  open high   
2022-05-11 12:15:00-04:00  195612.0     NaN  NaN NaN   NaN    NaN   NaN  NaN   
2022-05-11 12:20:00-04:00  281529.0     NaN  NaN NaN   NaN    NaN   NaN  NaN   

symbol                                      
                          low close volume  
2022-05-11 12:15:00-04:00 NaN   NaN    NaN  
2022-05-11 12:20:00-04:00 NaN   NaN    NaN

In [64]:
df_intraday.pt.operate(side="right")[:2]

symbol                           MSFT                                      \
                                 open        high         low       close   
2022-05-11 12:20:00-04:00  265.250000  265.522614  264.700012  264.809998   
2022-05-11 12:25:00-04:00  264.790009  266.299988  264.709991  266.200012   

symbol                              9988.HK                       AZN.L       \
                             volume    open high low close volume  open high   
2022-05-11 12:20:00-04:00  195612.0     NaN  NaN NaN   NaN    NaN   NaN  NaN   
2022-05-11 12:25:00-04:00  281529.0     NaN  NaN NaN   NaN    NaN   NaN  NaN   

symbol                                      
                          low close volume  
2022-05-11 12:20:00-04:00 NaN   NaN    NaN  
2022-05-11 12:25:00-04:00 NaN   NaN    NaN

The `PTIntrday` class can replicate the `side` behaviour directly with its `.pt.indexed_left` and `.pt.indexed_right` methods. (These methods are not available to other PT classes.)

In [65]:
df_intraday.pt.indexed_right[:2]

symbol                           MSFT                                      \
                                 open        high         low       close   
right                                                                       
2022-05-11 12:20:00-04:00  265.250000  265.522614  264.700012  264.809998   
2022-05-11 12:25:00-04:00  264.790009  266.299988  264.709991  266.200012   

symbol                              9988.HK                       AZN.L       \
                             volume    open high low close volume  open high   
right                                                                          
2022-05-11 12:20:00-04:00  195612.0     NaN  NaN NaN   NaN    NaN   NaN  NaN   
2022-05-11 12:25:00-04:00  281529.0     NaN  NaN NaN   NaN    NaN   NaN  NaN   

symbol                                      
                          low close volume  
right                                       
2022-05-11 12:20:00-04:00 NaN   NaN    NaN  
2022-05-11 12:25:00-04:00 NaN   NaN    NaN

**`close_only`** removes all columns except 'close' columns.

In [66]:
df_mult_days.pt.operate(close_only=True)

symbol,MSFT,9988.HK,AZN.L
"[2022-04-14, 2022-04-20)",285.299988,91.500000,10500.0
"[2022-04-20, 2022-04-25)",274.029999,86.650002,10280.0
"[2022-04-25, 2022-04-28)",283.220001,84.550003,10512.0
"[2022-04-28, 2022-05-03)",284.470001,102.099998,10688.0
"[2022-05-03, 2022-05-06)",277.350006,96.699997,10512.0
"[2022-05-06, 2022-05-11)",269.500000,86.000000,10246.0
"[2022-05-11, 2022-05-14)",262.309998,82.199997,10286.0


**`lose_single_symbol`** is only relevant if the table comprises a single symbol, in which case it removes the symbol level of the columns index.

In [67]:
df = df_daily.pt.operate(include="MSFT")
df[:2]

symbol            MSFT                                              
                  open        high         low       close    volume
2022-05-02  277.709991  284.940002  276.220001  284.470001  35151100
2022-05-03  283.959991  284.130005  280.149994  281.779999  25978600

In [68]:
df.pt.operate(lose_single_symbol=True)[:2]

,open,high,low,close,volume
2022-05-02,277.709991,284.940002,276.220001,284.470001,35151100
2022-05-03,283.959991,284.130005,280.149994,281.779999,25978600


Obviously, multiple options can be passed together to undertake various tidying operations with a single call.

In [69]:
df_intraday.pt.operate(
    exclude="AZN.L", fill="ffill", side="right", close_only=True
)

symbol,9988.HK,MSFT
2022-05-11 12:20:00-04:00,NaN,264.809998
2022-05-11 12:25:00-04:00,NaN,266.200012
2022-05-11 12:30:00-04:00,NaN,264.359985
2022-05-11 12:35:00-04:00,NaN,264.480011
2022-05-11 12:40:00-04:00,NaN,264.730011
...,...,...
2022-05-13 11:55:00-04:00,82.199997,261.859985
2022-05-13 12:00:00-04:00,82.199997,261.309998
2022-05-13 12:05:00-04:00,82.199997,261.600006
2022-05-13 12:10:00-04:00,82.199997,262.019989


## `PTIntraday` session mapping

PTIntraday offers various methods that map indices to sessions.

`.pt.sessions` returns a `pd.Series` that maps each indice to a session of a specfic calendar. The following example maps each indice to a session of the Hong Kong stock exchange.

In [70]:
df_intraday.pt.sessions(xhkg)

[2022-05-11 12:15:00, 2022-05-11 12:20:00)   2022-05-11
[2022-05-11 12:20:00, 2022-05-11 12:25:00)   2022-05-11
[2022-05-11 12:25:00, 2022-05-11 12:30:00)   2022-05-11
[2022-05-11 12:30:00, 2022-05-11 12:35:00)   2022-05-11
[2022-05-11 12:35:00, 2022-05-11 12:40:00)   2022-05-11
                                                ...    
[2022-05-13 11:50:00, 2022-05-13 11:55:00)   2022-05-13
[2022-05-13 11:55:00, 2022-05-13 12:00:00)   2022-05-13
[2022-05-13 12:00:00, 2022-05-13 12:05:00)   2022-05-13
[2022-05-13 12:05:00, 2022-05-13 12:10:00)   2022-05-13
[2022-05-13 12:10:00, 2022-05-13 12:15:00)   2022-05-13
Name: session, Length: 420, dtype: datetime64[ns]

If the left side of an indice does not represent a trading minute of the calendar then by default the indice is mapped to the more recent ("previous") session. `direction` can be passed as "next" to instead map such indices to the following session.

In [71]:
df_intraday.pt.sessions(xhkg, direction="next")

[2022-05-11 12:15:00, 2022-05-11 12:20:00)   2022-05-12
[2022-05-11 12:20:00, 2022-05-11 12:25:00)   2022-05-12
[2022-05-11 12:25:00, 2022-05-11 12:30:00)   2022-05-12
[2022-05-11 12:30:00, 2022-05-11 12:35:00)   2022-05-12
[2022-05-11 12:35:00, 2022-05-11 12:40:00)   2022-05-12
                                                ...    
[2022-05-13 11:50:00, 2022-05-13 11:55:00)   2022-05-16
[2022-05-13 11:55:00, 2022-05-13 12:00:00)   2022-05-16
[2022-05-13 12:00:00, 2022-05-13 12:05:00)   2022-05-16
[2022-05-13 12:05:00, 2022-05-13 12:10:00)   2022-05-16
[2022-05-13 12:10:00, 2022-05-13 12:15:00)   2022-05-16
Name: session, Length: 420, dtype: datetime64[ns]

The indices can also be mapped to a composite calendar, just pass `calendar` as an instance of `calutils.CompositeCalendar`.

In [72]:
df_intraday.pt.sessions(prices.cc)

[2022-05-11 12:15:00, 2022-05-11 12:20:00)   2022-05-11
[2022-05-11 12:20:00, 2022-05-11 12:25:00)   2022-05-11
[2022-05-11 12:25:00, 2022-05-11 12:30:00)   2022-05-11
[2022-05-11 12:30:00, 2022-05-11 12:35:00)   2022-05-11
[2022-05-11 12:35:00, 2022-05-11 12:40:00)   2022-05-11
                                                ...    
[2022-05-13 11:50:00, 2022-05-13 11:55:00)   2022-05-13
[2022-05-13 11:55:00, 2022-05-13 12:00:00)   2022-05-13
[2022-05-13 12:00:00, 2022-05-13 12:05:00)   2022-05-13
[2022-05-13 12:05:00, 2022-05-13 12:10:00)   2022-05-13
[2022-05-13 12:10:00, 2022-05-13 12:15:00)   2022-05-13
Name: session, Length: 420, dtype: datetime64[ns]

`.pt.session_column` adds the session mapping to the price table.

In [73]:
df_intraday.pt.session_column(prices.cc)[:2]

session        MSFT              \
                                              session        open        high   
[2022-05-11 12:15:00, 2022-05-11 12:20:00) 2022-05-11  265.250000  265.522614   
[2022-05-11 12:20:00, 2022-05-11 12:25:00) 2022-05-11  264.790009  266.299988   

                                                                              \
                                                   low       close    volume   
[2022-05-11 12:15:00, 2022-05-11 12:20:00)  264.700012  264.809998  195612.0   
[2022-05-11 12:20:00, 2022-05-11 12:25:00)  264.709991  266.200012  281529.0   

                                           9988.HK                        \
                                              open high low close volume   
[2022-05-11 12:15:00, 2022-05-11 12:20:00)     NaN  NaN NaN   NaN    NaN   
[2022-05-11 12:20:00, 2022-05-11 12:25:00)     NaN  NaN NaN   NaN    NaN   

                                           AZN.L                        
                                            open high low close volume  
[2022-05-11 12:15:00, 2022-05-11 12:20:00)   NaN  NaN NaN   NaN    NaN  
[2022-05-11 12:20:00, 2022-05-11 12:25:00)   NaN  NaN NaN   NaN    NaN

## `PTDailyIntradayComposite` methods

`PTDailyIntradayComposite` has a couple of methods unique to the class which provide for returning the daily and intraday parts of the table.

In [74]:
df_comp.pt.daily_part

symbol            MSFT                                                  \
                  open        high         low       close      volume   
2020-05-12  186.800003  187.039993  182.300003  182.509995  32038200.0   
2020-05-13  182.550003  184.050003  176.539993  179.750000  44711500.0   
2020-05-14  177.539993  180.690002  175.679993  180.529999  41873900.0   
2020-05-15  179.059998  187.059998  177.000000  183.160004  46610400.0   
2020-05-18  185.750000  186.199997  183.960007  184.910004  35264500.0   
...                ...         ...         ...         ...         ...   
2022-05-05  285.540009  286.350006  274.339996  277.350006  43260400.0   
2022-05-06  274.809998  279.250000  271.269989  274.730011  37748300.0   
2022-05-09  270.059998  272.359985  263.320007  264.579987  47726000.0   
2022-05-10  271.690002  273.750000  265.070007  269.500000  39336400.0   
2022-05-11  265.679993  271.359985  259.299988  260.549988  48975900.0   

symbol         9988.HK                                                  \
                  open        high         low       close      volume   
2020-05-12  198.899994  199.699997  197.199997  199.699997  14037259.0   
2020-05-13  195.500000  197.399994  194.300003  196.300003  26074457.0   
2020-05-14  194.500000  195.899994  194.100006  194.500000  19248894.0   
2020-05-15  195.000000  197.100006  194.100006  196.899994  16672799.0   
2020-05-18  198.300003  203.000000  197.399994  203.000000  26818890.0   
...                ...         ...         ...         ...         ...   
2022-05-05   99.500000  101.099998   96.199997   96.699997  27991225.0   
2022-05-06   92.050003   92.849998   89.300003   90.349998  50887668.0   
2022-05-09         NaN         NaN         NaN         NaN         NaN   
2022-05-10   83.500000   87.199997   83.150002   86.000000  64139439.0   
2022-05-11   83.699997   87.949997   83.599998   85.650002  63211009.0   

symbol        AZN.L                                            
               open         high      low    close     volume  
2020-05-12   8690.0   8871.00000   8672.0   8856.0  1636800.0  
2020-05-13   8800.0   9056.00000   8775.0   9004.0  2148708.0  
2020-05-14   8998.0   9027.00000   8705.0   8765.0  2375656.0  
2020-05-15   8740.0   8796.00000   8536.0   8671.0  2410210.0  
2020-05-18   8849.0   8898.00000   8729.0   8790.0  2098697.0  
...             ...          ...      ...      ...        ...  
2022-05-05  10532.0  10632.00000  10478.0  10512.0  2501672.0  
2022-05-06  10464.0  10474.00000  10222.0  10322.0  2591818.0  
2022-05-09  10372.0  10462.47168  10118.0  10140.0  2216925.0  
2022-05-10  10166.0  10338.00000  10014.0  10246.0  1759026.0  
2022-05-11  10210.0  10262.00000   9901.0  10026.0  4552005.0  

[518 rows x 15 columns]

In [75]:
df_comp.pt.intraday_part

symbol                                            MSFT              \
                                                  open        high   
[2022-05-12 01:30:00, 2022-05-12 01:31:00)         NaN         NaN   
[2022-05-12 01:31:00, 2022-05-12 01:32:00)         NaN         NaN   
[2022-05-12 01:32:00, 2022-05-12 01:33:00)         NaN         NaN   
[2022-05-12 01:33:00, 2022-05-12 01:34:00)         NaN         NaN   
[2022-05-12 01:34:00, 2022-05-12 01:35:00)         NaN         NaN   
...                                                ...         ...   
[2022-05-12 19:52:00, 2022-05-12 19:53:00)  254.529999  254.630005   
[2022-05-12 19:53:00, 2022-05-12 19:54:00)  254.550003  254.559998   
[2022-05-12 19:54:00, 2022-05-12 19:55:00)  254.029999  254.335007   
[2022-05-12 19:55:00, 2022-05-12 19:56:00)  254.000000  254.309998   
[2022-05-12 19:56:00, 2022-05-12 19:57:00)  254.080002  254.899994   

symbol                                                                        \
                                                   low       close    volume   
[2022-05-12 01:30:00, 2022-05-12 01:31:00)         NaN         NaN       NaN   
[2022-05-12 01:31:00, 2022-05-12 01:32:00)         NaN         NaN       NaN   
[2022-05-12 01:32:00, 2022-05-12 01:33:00)         NaN         NaN       NaN   
[2022-05-12 01:33:00, 2022-05-12 01:34:00)         NaN         NaN       NaN   
[2022-05-12 01:34:00, 2022-05-12 01:35:00)         NaN         NaN       NaN   
...                                                ...         ...       ...   
[2022-05-12 19:52:00, 2022-05-12 19:53:00)  254.279999  254.529999  219682.0   
[2022-05-12 19:53:00, 2022-05-12 19:54:00)  253.740005  254.029999  209035.0   
[2022-05-12 19:54:00, 2022-05-12 19:55:00)  253.259995  254.020004  277838.0   
[2022-05-12 19:55:00, 2022-05-12 19:56:00)  253.820007  254.070007  289084.0   
[2022-05-12 19:56:00, 2022-05-12 19:57:00)  254.080002  254.800003  352067.0   

symbol                                        9988.HK                        \
                                                 open       high        low   
[2022-05-12 01:30:00, 2022-05-12 01:31:00)  81.849998  82.550003  81.849998   
[2022-05-12 01:31:00, 2022-05-12 01:32:00)  82.500000  82.550003  82.199997   
[2022-05-12 01:32:00, 2022-05-12 01:33:00)  82.300003  82.500000  82.000000   
[2022-05-12 01:33:00, 2022-05-12 01:34:00)  82.050003  82.349998  81.900002   
[2022-05-12 01:34:00, 2022-05-12 01:35:00)  82.300003  82.300003  81.599998   
...                                               ...        ...        ...   
[2022-05-12 19:52:00, 2022-05-12 19:53:00)        NaN        NaN        NaN   
[2022-05-12 19:53:00, 2022-05-12 19:54:00)        NaN        NaN        NaN   
[2022-05-12 19:54:00, 2022-05-12 19:55:00)        NaN        NaN        NaN   
[2022-05-12 19:55:00, 2022-05-12 19:56:00)        NaN        NaN        NaN   
[2022-05-12 19:56:00, 2022-05-12 19:57:00)        NaN        NaN        NaN   

symbol                                                           AZN.L       \
                                                close     volume  open high   
[2022-05-12 01:30:00, 2022-05-12 01:31:00)  82.550003        0.0   NaN  NaN   
[2022-05-12 01:31:00, 2022-05-12 01:32:00)  82.250000   641178.0   NaN  NaN   
[2022-05-12 01:32:00, 2022-05-12 01:33:00)  82.099998  1475000.0   NaN  NaN   
[2022-05-12 01:33:00, 2022-05-12 01:34:00)  82.250000   776810.0   NaN  NaN   
[2022-05-12 01:34:00, 2022-05-12 01:35:00)  81.750000   938189.0   NaN  NaN   
...                                               ...        ...   ...  ...   
[2022-05-12 19:52:00, 2022-05-12 19:53:00)        NaN        NaN   NaN  NaN   
[2022-05-12 19:53:00, 2022-05-12 19:54:00)        NaN        NaN   NaN  NaN   
[2022-05-12 19:54:00, 2022-05-12 19:55:00)        NaN        NaN   NaN  NaN   
[2022-05-12 19:55:00, 2022-05-12 19:56:00)        NaN        NaN   NaN  NaN   
[2022-05-12 19:56:00, 2022-05-12 19:57:00)        NaN        NaN   NaN  